In [1]:
import numpy as np
import plotly.graph_objects as go
from plotly.offline import plot

# Generate 40 stock charts and store them as individual HTML div variables
for i in range(1, 41):
    # Generate sample data
    days = np.arange(100)
    stock_prices = np.cumsum(np.random.randn(100)) + 100
    
    # Create and configure figure
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=days, 
        y=stock_prices,
        mode='lines',
        line=dict(color='#30415f', width=2),
        name=f'Stock {i}'
    ))
    fig.update_layout(
        title=f'Stock Graph {i}',
        xaxis_title='Days',
        yaxis_title='Stock Price',
        plot_bgcolor='white',
        font=dict(family='Montserrat'),
        showlegend=False
    )
    
    # Create dynamic variable name and store HTML
    globals()[f'graph_html{i}'] = plot(
        fig, 
        output_type='div', 
        include_plotlyjs='cdn' if i == 1 else False
    )


In [1]:
import math
import pandas as pd
from scipy.stats import zscore
import numpy as np
import matplotlib.pyplot as plt
import pathlib
from datetime import datetime, date, timedelta
from xbbg import blp
import blpapi
import os
import win32com.client as win32
from sklearn.preprocessing import StandardScaler
from scipy.stats.mstats import winsorize
import plotly.express as px
import plotly.figure_factory as ff
import plotly.graph_objects as go
from plotly.io import to_html
from plotly.subplots import make_subplots
import yfinance as yf
import pandas as pd
import numpy as np
import math
import datetime
import statsmodels.api as sm
import plotly.express as px
import plotly.figure_factory as ff
from plotly.offline import plot
import scipy.optimize as spop
from scipy.stats import t
from scipy.stats import norm
from statsmodels.regression.rolling import RollingOLS
from datetime import datetime
import requests
from bs4 import BeautifulSoup
import re
import matplotlib.colors as mcolors
cmap = mcolors.LinearSegmentedColormap.from_list(
    "soft_RdYlGn", ["lightcoral", "white", "lightgreen"]
)

def winsorize_df(df, lower_limit=0.01, upper_limit=0.99):
    return df.apply(lambda col: winsorize(col, limits=(lower_limit, 1-upper_limit)))

def timeseriesplotting(valuation_df, valuation_metric, country):
    country_df = valuation_df[country]
    data = country_df[valuation_metric].dropna()  
    mean_val = data.median()
    std_val = data.std()
    fig = px.line(data, width = 1300, height = 600, title = f'{valuation_metric} : {country}')
    fig.update_traces(line=dict(color='#30415f'))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val] * len(data), mode='lines', name='Mean', line=dict(dash='solid', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val + std_val] * len(data), mode='lines', name='+ 1 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val - std_val] * len(data), mode='lines', name='- 1 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val - (std_val*2)] * len(data), mode='lines', name='- 2 Std', line=dict(dash='dot', color = 'grey')))
    fig.add_trace(go.Scatter(x=data.index, y=[mean_val + (std_val*2)] * len(data), mode='lines', name='+ 1 Std', line=dict(dash='dot', color = 'grey')))

    fig.update_layout(font=dict(family="Montserrat", size=13), title=dict(font=dict(family="Montserrat", size=16)), plot_bgcolor='white')

    if valuation_metric == valuation_list[0]:
        fig.update_layout(title=dict(text=f'<b><span style="color:darkcyan;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[1]:
        fig.update_layout(title=dict(text=f'<b><span style="color:coral;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[2]:
        fig.update_layout(title=dict(text=f'<b><span style="color:crimson;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))
    elif valuation_metric == valuation_list[3]:
        fig.update_layout(title=dict(text=f'<b><span style="color:cadetblue;">{valuation_metric}</span> : {country}</b>',font=dict(size=21, family='Montserrat', color='black')))

    fig.update_xaxes(tickangle=45, title_text="", tickfont=dict(size=10))
    fig.update_yaxes(title_text=f'{valuation_metric}')

    return fig

def per_valuation_plotter(name_list, valuation_list, valuation_metric, winsorized_add_composite):
    total_plots = len(name_list)
    rows = math.ceil(total_plots / 2)  # Round up to handle odd numbers

    # Create a subplot figure with the specified rows and 2 columns
    fig = make_subplots(rows=rows, cols=3, subplot_titles=[f"{valuation_metric} - {j}" for j in name_list])

    # Loop over each region and add the corresponding plot to the subplot
    for idx, j in enumerate(name_list):
        row = (idx // 3) + 1  # Calculate the row
        col = (idx % 3) + 1   # Calculate the column

        # Generate the plot for the region with the specified valuation metric
        initial = timeseriesplotting(winsorized_add_composite, valuation_metric, j)
        
        # Add each trace in `initial` to the subplot, hiding the legend
        for trace in initial.data:
            trace.showlegend = False
            fig.add_trace(trace, row=row, col=col)

        # Apply individual title and font color based on `valuation_metric` for each subplot
        title_color = 'black'  # Default color
        if valuation_metric == valuation_list[0]:
            title_color = 'darkcyan'
        elif valuation_metric == valuation_list[1]:
            title_color = 'coral'
        elif valuation_metric == valuation_list[2]:
            title_color = 'crimson'
        elif valuation_metric == valuation_list[3]:
            title_color = 'cadetblue'
        elif valuation_metric == valuation_list[4]:
            title_color = 'purple'
        elif valuation_metric == valuation_list[5]:
            title_color = 'green'

        # Update title for each subplot individually
        fig.layout.annotations[idx].update(
            text=f'<b><span style="color:{title_color};">{valuation_metric}</span> : {j}</b>',
            font=dict(size=16, family='Montserrat')
        )

    # Update layout with global settings for font and plot background color
    fig.update_layout(
        height=400 * rows, width=1500,
        font=dict(family="Montserrat", size=13),
        plot_bgcolor='white'
    )

    # Apply X and Y axis formatting to all subplots
    fig.update_xaxes(tickangle=45, title_text="", tickfont=dict(size=10))
    fig.update_yaxes(title_text=f'{valuation_metric}')

    return fig

def clean_data_after_bloomberg(multi_level_df, names_of_the_category, valuation_list):
    multi_level_df.columns = multi_level_df.columns.set_levels([names_of_the_category, valuation_list])
    multi_level_df.index = pd.to_datetime(multi_level_df.index)
    multi_level_df.index = multi_level_df.index.strftime('%b-%Y')
    multi_level_df_winsorized = winsorize_df(multi_level_df, 0.01, 0.99)

    for i in multi_level_df_winsorized.columns.levels[0]:
        country = multi_level_df_winsorized[i]
        z_score_per_val = (( country - country.mean() ) / country.std() )   # seperate Z_score DF
        without_PE = z_score_per_val.drop('Forward PE', axis = 1)       
        valuation_z_score = without_PE.mean(axis=1)                          # Z-score w/o F-PE
        multi_level_df_winsorized[(i,'Valuation Composite')] = valuation_z_score            # add that main
    multi_level_df_winsorized_add_composite = multi_level_df_winsorized.replace(0, np.nan)
    
    # if names_of_the_category == regionals_name:
    #     valuation_list.append('Valuation Composite')
    valuation_list.append('Valuation Composite')

    return multi_level_df_winsorized_add_composite, valuation_list, z_score_per_val

def generate_graphs(values_composite, names, prefix):
    for valuation_metric in values_composite[1]:
        var_name = f"{prefix}_{valuation_metric.replace(' ', '_').replace('/', '_')}"  # Fix naming
        graph_list = [plot(timeseriesplotting(values_composite[0], valuation_metric, name), 
                           output_type='div', include_plotlyjs='cdn') for name in names]

        globals()[var_name] = graph_list  # Directly create variables
        print(f"{var_name} = {graph_list}")  # Print for visibility

def simp_graph_performance(major_equity_ytd, title):

    if isinstance(major_equity_ytd, pd.Series):

        # Use color_discrete_sequence for direct color assignment
        major_equity_ytd_graph = px.line(
            major_equity_ytd, width = 1650,
            color_discrete_sequence = [
            "#30415f",  # Flame 30415f
            "#f3a712",  # Amber
            "#87b1a1",  # Sage
            "#5ac5fe",  # 
            "#a8c686",  # Pistachio
            "#a0a197",  # Slate
            "#e4572e",  # Midnight
            "#2337C6",  # 
            "#B7B1B0",  # Taupe
            "#778BA5",  # Cadet Blue
            "#990000"   # 
        ]

        )
        major_equity_ytd_graph.update_layout(
            font_family="Montserrat",
            title={
                "text": title,
                "font": {"size": 22}
            })

        major_equity_ytd_graph_html = plot(major_equity_ytd_graph, output_type='div', include_plotlyjs='cdn')

    elif isinstance(major_equity_ytd, pd.DataFrame):

        # Use color_discrete_sequence for direct color assignment
        major_equity_ytd_graph = px.line(
            major_equity_ytd, width = 1250,
            color_discrete_sequence = [
            "#30415f",  # 
            "#DDDDDD",  # Amber
            "#DDDDDD",  # Sage
            "#DDDDDD",  # 
        ]

        )
        major_equity_ytd_graph.update_layout(
            font_family="Montserrat",
            title={
                "text": title,
                "font": {"size": 22}
            })

        major_equity_ytd_graph_html = plot(major_equity_ytd_graph, output_type='div', include_plotlyjs='cdn')
        
    return major_equity_ytd_graph_html

def graph_performance(major_equity_ytd, title):
    major_equity_ytd_graph = px.line(major_equity_ytd)
    major_equity_ytd_graph.update_layout(
        font_family="Montserrat",
        title={
            "text": title,
            "font": {"size": 22}
        },
        xaxis_title="",
        yaxis_title="Price",
        yaxis=dict(side="left",
            title="Price",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            gridcolor="#ECECEC",
            linecolor="#ECECEC",
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        width=1250,
        height=600,
        legend=dict(
            orientation="h",
            y=-0.075,
            x=0.5,
            xanchor="center"
        )
    )

    color_palette = [
        "#30415f",  # Flame 30415f
        "#f3a712",  # Amber
        "#87b1a1",  # Sage
        "#5ac5fe",  # 
        "#a8c686",  # Pistachio
        "#a0a197",  # Slate
        "#e4572e",  # Midnight
        "#2337C6",  # 
        "#B7B1B0",  # Taupe
        "#778BA5",  # Cadet Blue
        "#990000"   # 
    ]

    for i, trace in enumerate(major_equity_ytd_graph.data):
        trace.update(line=dict(color=color_palette[i % len(color_palette)]))

    major_equity_ytd_graph_html = plot(major_equity_ytd_graph, output_type='div', include_plotlyjs='cdn')
    return major_equity_ytd_graph_html


def cross_sectional_current_table_maker(factor_vals_winsorized_add_composite):
    """
    Creates an HTML table styled like the existing dashboard tables from a DataFrame.
    Replaces the Plotly table generation.
    """
    if factor_vals_winsorized_add_composite[0].columns.levels[0][0] == 'Value':
        xs_factor_current = factor_vals_winsorized_add_composite[0].iloc[-1].reset_index().pivot(
            index='level_1',  # Set 'level_1' as the rows
            columns='level_0',  # Set 'level_0' as the columns
            values=factor_vals_winsorized_add_composite[0].iloc[-1].name
        ).T.reset_index().rename({'level_0': 'Global Factors'}, axis=1).round(1).sort_values('Forward PE', ascending=False)
    else:
        xs_factor_current = factor_vals_winsorized_add_composite[0].iloc[-1].reset_index().pivot(
            index='level_1',  # Set 'level_1' as the rows
            columns='level_0',  # Set 'level_0' as the columns
            values=factor_vals_winsorized_add_composite[0].iloc[-1].name
        ).T.reset_index().rename({'level_0': 'Region'}, axis=1).round(1).sort_values('Forward PE', ascending=False)

    df = xs_factor_current.copy() # Create a copy to avoid modifying the original

    # Define styles for the table
    styles = [
        {'selector': 'td:hover', 'props': [('background-color', '#30415f')]},
        {'selector': 'th:not(.index_name)', 'props': [('background-color', '#30415f'), ('color', 'white'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('font-size', '14px'), ('text-align', 'center'), ('width', '120px'), ('border', '1px solid #ddd')]},
        {'selector': 'th', 'props': [('text-align', 'left'), ('border', '1px solid #ddd')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse')]}
    ]

    # Add color gradient to Valuation Composite column
    styled_table = (df.style
        .set_table_styles(styles)
        .format(precision=1)
        .background_gradient(
            subset=['Valuation Composite'],  # Target specific column
            cmap='RdYlGn_r',  # Red-Yellow-Green reversed (green high, red low)
            axis=0  # Apply per-column
        )
        .hide_index()
        .to_html())

    return styled_table

Bond Futures (DEC 2025):

In [ ]:
number_of_cuts = ['US0ANM DEC2025 Index','AU0ANM DEC2025 Index','EZ0BNM DEC2025 Index']
custom_colors = ['#003366', '#669bbc', '#f3a712', '#e4572e', '#87b1a1', '#a0a197', '#22a6a3', '#3454d1','#78adf9']
data = blp.bdh(tickers = number_of_cuts, flds= 'px_last', start_date= '31-01-2024')
names = blp.bdp(number_of_cuts, flds = 'Country')
data.columns = ['US','AU','EU']

rate_futures = px.line(-data.iloc[20:])  # Replace -data with data if that's the correct dataset

rate_futures.update_layout(
    font_family="Montserrat",
    title={
        "text": "Number of Rate Cuts by December 2025",
        "font": {"size": 22}  # Title font size and bold weight
    },
    xaxis_title="",
    yaxis_title="Number of Cuts/Hikes",
    yaxis=dict(side="right",
        title="Number of Cuts/Hikes",
        titlefont=dict(color="black"),  # Left y-axis label color
        tickfont=dict(color="black"),  # Left y-axis tick labels color
        gridcolor="lightgray",
        linecolor="gray",
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(gridcolor="lightgray", linecolor="gray"),
    width=950,
    height=600,
    legend=dict(
        orientation="h",  # Horizontal layout
        y=-0.075,           # Position below the chart
        x=0.5,
        xanchor="center"
    )
)

# Updating trace colors (assume 3 traces: blue, red, green)
colors = ["#020035", "#4682B4", "orange"]
for i, trace in enumerate(rate_futures.data):
    trace.update(line=dict(color=colors[i % len(colors)]))  # Cycle colors if more than 3 traces

rate_futures_html = plot(rate_futures, output_type='div', include_plotlyjs='cdn')

Yield curve (old innva)

In [8]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.io as pio
from datetime import datetime, date, timedelta
from xbbg import blp

# ---------------------------
# Set up tickers and timeframes
# ---------------------------
UStickers = ['USGG3M Index','USGG6M Index', 'USGG12M Index', 
             'USGG2YR Index', 'USGG3Y Index', 'USGG5YR Index', 
             'USGG7Y Index','USGG10Y Index','USGG20Y Index','USGG30Y Index']

EUtickers = ['GTEUR3M Govt','GTEUR6M Govt', 'GTEUR1Y Govt', 
             'GTEUR2Y Govt', 'GTEUR3Y Govt', 'GTEUR5Y Govt', 
             'GTEUR7Y Govt','GTEUR10Y Govt','GTEUR20Y Govt','GTEUR30Y Govt']

AUtickers = ['GTAUD3M Govt', 'GTAUD1Y Govt', 
             'GTAUD2Y Govt', 'GTAUD3Y Govt', 'GTAUD5Y Govt', 
             'GTAUD7Y Govt','GTAUD10Y Govt','GTAUD20Y Govt','GTAUD30Y Govt']

today = datetime.today()
three = today - timedelta(days=3)
week  = today - timedelta(days=7)
month = today - timedelta(days=30)
formatted_date = today.strftime("%Y-%m-%d")

# ---------------------------
# Helper function: create a Plotly line chart
# ---------------------------
def create_line_chart(df_long, x_col, y_col, color_col, title, width=900, height=500, dash_styles=None):
    fig = px.line(df_long, x=x_col, y=y_col, color=color_col, 
                  title=title, width=width, height=height)
    
    # Customize colors and appearance
    # No selector needed; apply color to all lines initially
    fig.update_traces(line=dict(color='#30415f'))

    if dash_styles:
        for i, trace in enumerate(fig.data):
            # Apply dash styles and light gray color ONLY to non-main lines
            if trace.name != df_long[color_col].unique()[0]: #df_long[color_col].unique()[0] is a way to identify 'main' line
                trace.line.dash = dash_styles[i] if i < len(dash_styles) else 'solid'
                trace.line.color = 'lightgray'

    fig.update_layout(
        font=dict(family="Montserrat"),
        plot_bgcolor='white',
        xaxis=dict(showgrid=False),
        yaxis=dict(showgrid=False)
    )
    return fig

# ---------------------------
# US Yield Curve Chart
# ---------------------------
UStoday   = blp.bdh(tickers=UStickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)
USthree   = blp.bdh(tickers=UStickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=three).tail(1)
USweek    = blp.bdh(tickers=UStickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=week).tail(1)
USmonth   = blp.bdh(tickers=UStickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=month).tail(1)

UScurve = pd.concat([UStoday, USthree, USweek, USmonth], axis=0)
UScurve.columns = UScurve.columns.droplevel(1)
UScurve = UScurve.rename(index={UScurve.index[0]: 'Today', 
                                UScurve.index[1]: 'Three Days', 
                                UScurve.index[2]: 'Week', 
                                UScurve.index[3]: 'Month'})
UScurve = UScurve.rename(columns={
    UScurve.columns[0]: '3m',
    UScurve.columns[1]: '6m',
    UScurve.columns[2]: '12m',
    UScurve.columns[3]: '2y',
    UScurve.columns[4]: '3y',
    UScurve.columns[5]: '5y',
    UScurve.columns[6]: '7y',
    UScurve.columns[7]: '10y',
    UScurve.columns[8]: '20y',
    UScurve.columns[9]: '30y'
})
# Transpose so maturities become the x-axis
UScurvechart = UScurve.transpose()
df_us = UScurvechart.reset_index().rename(columns={'index': 'Maturity'})
df_us_long = df_us.melt(id_vars='Maturity', var_name='TimeFrame', value_name='Yield')

dash_styles = ['solid', 'dash', 'dash', 'dash']
fig_us = create_line_chart(df_us_long, 'Maturity', 'Yield', 'TimeFrame', 
                           f'US Yield Curve {date.today()}', dash_styles=dash_styles)
US_chart_html = pio.to_html(fig_us, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# EU Yield Curve Chart
# ---------------------------
EUtoday = blp.bdh(tickers=EUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)
EUthree = blp.bdh(tickers=EUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=three).tail(1)
EUweek  = blp.bdh(tickers=EUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=week).tail(1)
EUmonth = blp.bdh(tickers=EUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=month).tail(1)

EUcurve = pd.concat([EUtoday, EUthree, EUweek, EUmonth], axis=0)
EUcurve.columns = EUcurve.columns.droplevel(1)
EUcurve = EUcurve.rename(index={EUcurve.index[0]: 'Today', 
                                EUcurve.index[1]: 'Three Days', 
                                EUcurve.index[2]: 'Week', 
                                EUcurve.index[3]: 'Month'})
EUcurve = EUcurve.rename(columns={
    EUcurve.columns[0]: '3m',
    EUcurve.columns[1]: '6m',
    EUcurve.columns[2]: '12m',
    EUcurve.columns[3]: '2y',
    EUcurve.columns[4]: '3y',
    EUcurve.columns[5]: '5y',
    EUcurve.columns[6]: '7y',
    EUcurve.columns[7]: '10y',
    EUcurve.columns[8]: '20y',
    EUcurve.columns[9]: '30y'
})
EUcurvechart = EUcurve.transpose()
df_eu = EUcurvechart.reset_index().rename(columns={'index': 'Maturity'})
df_eu_long = df_eu.melt(id_vars='Maturity', var_name='TimeFrame', value_name='Yield')

fig_eu = create_line_chart(df_eu_long, 'Maturity', 'Yield', 'TimeFrame', 
                           f'EU Yield Curve {date.today()}', dash_styles=dash_styles)
EU_chart_html = pio.to_html(fig_eu, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# AU Yield Curve Chart
# ---------------------------
AUtoday = blp.bdh(tickers=AUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)
AUthree = blp.bdh(tickers=AUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=three).tail(1)
AUweek  = blp.bdh(tickers=AUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=week).tail(1)
AUmonth = blp.bdh(tickers=AUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=month).tail(1)

AUcurve = pd.concat([AUtoday, AUthree, AUweek, AUmonth], axis=0)
AUcurve.columns = AUcurve.columns.droplevel(1)
AUcurve = AUcurve.rename(index={AUcurve.index[0]: 'Today', 
                                AUcurve.index[1]: 'Three Days', 
                                AUcurve.index[2]: 'Week', 
                                AUcurve.index[3]: 'Month'})
AUcurve = AUcurve.rename(columns={
    AUcurve.columns[0]: '3m',
    AUcurve.columns[1]: '12m',  # Note: a 6m value is inserted later.
    AUcurve.columns[2]: '2y',
    AUcurve.columns[3]: '3y',
    AUcurve.columns[4]: '5y',
    AUcurve.columns[5]: '7y',
    AUcurve.columns[6]: '10y',
    AUcurve.columns[7]: '20y',
    AUcurve.columns[8]: '30y'
})
AUcurvechart = AUcurve.transpose()
df_au = AUcurvechart.reset_index().rename(columns={'index': 'Maturity'})
df_au_long = df_au.melt(id_vars='Maturity', var_name='TimeFrame', value_name='Yield')

fig_au = create_line_chart(df_au_long, 'Maturity', 'Yield', 'TimeFrame', 
                           f'AU Yield Curve {date.today()}', dash_styles=dash_styles)
AU_chart_html = pio.to_html(fig_au, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# AusBond Composite OAS
# ---------------------------
Aus_comp = blp.bdh(tickers='BACM0 Index', flds=['OAS_SPREAD_MID'], start_date='2015-01-01', end_date=today)
Aus_comp.columns = Aus_comp.columns.droplevel(1)
average = Aus_comp['BACM0 Index'].mean()
stddev  = Aus_comp['BACM0 Index'].std()
Aus_comp['Average'] = average
Aus_comp['+1 STD'] = average + stddev
Aus_comp['-1 STD'] = average - stddev

df_aus_comp = Aus_comp.reset_index().rename(columns={'index': 'Date'})
df_aus_comp_long = df_aus_comp.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
dash_styles_comp = ['solid', 'dash', 'dash', 'dash']
fig_aus_comp = create_line_chart(df_aus_comp_long, 'Date', 'OAS_Spread', 'Legend', 
                                 f'Ausbond Composite 0+Yr Spread {date.today()}', dash_styles=dash_styles_comp)
Aus_comp_chart_html = pio.to_html(fig_aus_comp, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# AusBond Credit OAS
# ---------------------------
Aus_cred = blp.bdh(tickers='BACR0 Index', flds=['OAS_SPREAD_MID'], start_date='2015-01-01', end_date=today)
Aus_cred.columns = Aus_cred.columns.droplevel(1)
average = Aus_cred['BACR0 Index'].mean()
stddev  = Aus_cred['BACR0 Index'].std()
Aus_cred['Average'] = average
Aus_cred['+1 STD'] = average + stddev
Aus_cred['-1 STD'] = average - stddev

df_aus_cred = Aus_cred.reset_index().rename(columns={'index': 'Date'})
df_aus_cred_long = df_aus_cred.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
fig_aus_cred = create_line_chart(df_aus_cred_long, 'Date', 'OAS_Spread', 'Legend', 
                                 f'Ausbond Credit 0+Yr Spread {date.today()}', dash_styles=dash_styles_comp)
Aus_cred_chart_html = pio.to_html(fig_aus_cred, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# AusBond Credit FRN
# ---------------------------
Aus_FRN = blp.bdh(tickers='BAFRN0 Index', flds=['OAS_SPREAD_MID'], start_date='2015-01-01', end_date=today)
Aus_FRN.columns = Aus_FRN.columns.droplevel(1)
average = Aus_FRN['BAFRN0 Index'].mean()
stddev  = Aus_FRN['BAFRN0 Index'].std()
Aus_FRN['Average'] = average
Aus_FRN['+1 STD'] = average + stddev
Aus_FRN['-1 STD'] = average - stddev

df_aus_frn = Aus_FRN.reset_index().rename(columns={'index': 'Date'})
df_aus_frn_long = df_aus_frn.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
fig_aus_frn = create_line_chart(df_aus_frn_long, 'Date', 'OAS_Spread', 'Legend', 
                                f'Ausbond Credit FRN 0+Yr Spread {date.today()}', dash_styles=dash_styles_comp)
Aus_FRN_chart_html = pio.to_html(fig_aus_frn, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# US Aggregate Corporate
# ---------------------------
US_corp = blp.bdh(tickers='LUACOAS Index', flds=['PX_LAST'], start_date='2015-01-01', end_date=today)
US_corp.columns = US_corp.columns.droplevel(1)
average = US_corp['LUACOAS Index'].mean()
stddev  = US_corp['LUACOAS Index'].std()
US_corp['Average'] = average
US_corp['+1 STD'] = average + stddev
US_corp['-1 STD'] = average - stddev

df_us_corp = US_corp.reset_index().rename(columns={'index': 'Date'})
df_us_corp_long = df_us_corp.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
fig_us_corp = create_line_chart(df_us_corp_long, 'Date', 'OAS_Spread', 'Legend', 
                                f'US Agg Corporate OAS Spread {date.today()}', dash_styles=dash_styles_comp)
US_corp_chart_html = pio.to_html(fig_us_corp, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# US Aggregate Credit
# ---------------------------
US_cred = blp.bdh(tickers='LUCROAS Index', flds=['PX_LAST'], start_date='2015-01-01', end_date=today)
US_cred.columns = US_cred.columns.droplevel(1)
average = US_cred['LUCROAS Index'].mean()
stddev  = US_cred['LUCROAS Index'].std()
US_cred['Average'] = average
US_cred['+1 STD'] = average + stddev
US_cred['-1 STD'] = average - stddev

df_us_cred = US_cred.reset_index().rename(columns={'index': 'Date'})
df_us_cred_long = df_us_cred.melt(id_vars='Date', var_name='Legend', value_name='OAS_Spread')
fig_us_cred = create_line_chart(df_us_cred_long, 'Date', 'OAS_Spread', 'Legend', 
                                f'US Agg Credit OAS Spread {date.today()}', dash_styles=dash_styles_comp)
US_cred_chart_html = pio.to_html(fig_us_cred, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# Global Yield Curves (Combined)
# ---------------------------
UStoday1 = blp.bdh(tickers=UStickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)
EUtoday1 = blp.bdh(tickers=EUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)
AUtoday1 = blp.bdh(tickers=AUtickers, flds=['PX_LAST'], start_date='2022-01-01', end_date=today).tail(1)

UStoday1 = UStoday1.rename(index={UStoday1.index[0]: 'US Yield Curve'})
UStoday1.columns = UStoday1.columns.droplevel(1)
UStoday1 = UStoday1.rename(columns={
    UStoday1.columns[0]: '3m',
    UStoday1.columns[1]: '6m',
    UStoday1.columns[2]: '12m',
    UStoday1.columns[3]: '2y',
    UStoday1.columns[4]: '3y',
    UStoday1.columns[5]: '5y',
    UStoday1.columns[6]: '7y',
    UStoday1.columns[7]: '10y',
    UStoday1.columns[8]: '20y',
    UStoday1.columns[9]: '30y'
})

EUtoday1 = EUtoday1.rename(index={EUtoday1.index[0]: 'EU Yield Curve'})
EUtoday1.columns = EUtoday1.columns.droplevel(1)
EUtoday1 = EUtoday1.rename(columns={
    EUtoday1.columns[0]: '3m',
    EUtoday1.columns[1]: '6m',
    EUtoday1.columns[2]: '12m',
    EUtoday1.columns[3]: '2y',
    EUtoday1.columns[4]: '3y',
    EUtoday1.columns[5]: '5y',
    EUtoday1.columns[6]: '7y',
    EUtoday1.columns[7]: '10y',
    EUtoday1.columns[8]: '20y',
    EUtoday1.columns[9]: '30y'
})

AUtoday1 = AUtoday1.rename(index={AUtoday1.index[0]: 'AU Yield Curve'})
AUtoday1.columns = AUtoday1.columns.droplevel(1)
AUtoday1 = AUtoday1.rename(columns={
    AUtoday1.columns[0]: '3m',
    AUtoday1.columns[1]: '12m',
    AUtoday1.columns[2]: '2y',
    AUtoday1.columns[3]: '3y',
    AUtoday1.columns[4]: '5y',
    AUtoday1.columns[5]: '7y',
    AUtoday1.columns[6]: '10y',
    AUtoday1.columns[7]: '20y',
    AUtoday1.columns[8]: '30y'
})
# Manually add a 6m column for the AU curve
AUtoday1.insert(1, '6m', (AUtoday1['3m'] + AUtoday1['12m']) / 2)

Global = pd.concat([UStoday1, EUtoday1, AUtoday1], axis=0, join='outer')
Globalgraph = Global.transpose()
df_global = Globalgraph.reset_index().rename(columns={'index': 'Maturity'})
df_global_long = df_global.melt(id_vars='Maturity', var_name='Curve', value_name='Yield')
fig_global = create_line_chart(df_global_long, 'Maturity', 'Yield', 'Curve', 
                               f'Global Yield Curves {date.today()}')
# All curves with solid lines
for trace in fig_global.data:
    trace.line.dash = 'solid'
Global_chart_html = pio.to_html(fig_global, full_html=False, include_plotlyjs='cdn')


# ---------------------------
# Global Yield Curves (AUD Hedged)
# ---------------------------
# Retrieve FX data and calculate forward adjustments
AUDUSDspot = blp.bdh(tickers='AUDUSD Curncy', flds=['PX_LAST'], start_date='2023-04-01', end_date=today).tail(1)
AUDEURspot = blp.bdh(tickers='AUDEUR Curncy', flds=['PX_LAST'], start_date='2023-04-01', end_date=today).tail(1)
EURAUDspot = blp.bdh(tickers='EURAUD Curncy', flds=['PX_LAST'], start_date='2023-04-01', end_date=today).tail(1)

AUDUSDforward = blp.bdh(tickers='AUD1M BGN Curncy', flds=['PX_LAST'], start_date='2023-04-01', end_date=today).tail(1)
AUDUSDforward = (AUDUSDforward * 12) / 10000
EURAUDforward = blp.bdh(tickers='ADEU1M BGN Curncy', flds=['PX_LAST'], start_date='2023-04-01', end_date=today).tail(1)
EURAUDforward = (EURAUDforward * 12) / 10000

AUDEURimplied = 1 / (EURAUDspot.values[0] + EURAUDforward.values[0])
AUDEURforward = (AUDEURimplied - AUDEURspot.values[0])

# Build adjusted yield curves for US and EU
UStodayfx = {}
for col in UStoday1.columns:
    # Adjust yields: (yield/100) - [forward/(1+yield/100)]*spot
    val = (UStoday1[col] / 100) - AUDUSDforward.values[0] / (1 + (UStoday1[col] / 100)) * AUDUSDspot.values[0]
    UStodayfx[col] = val
UStodayfx = pd.DataFrame(UStodayfx).mul(100).round(3)

EUtodayfx = {}
for col in EUtoday1.columns:
    val = (EUtoday1[col] / 100) - AUDEURforward / (1 + (EUtoday1[col] / 100)) * AUDEURspot.values[0]
    EUtodayfx[col] = val
EUtodayfx = pd.DataFrame(EUtodayfx).mul(100).round(3)

Globalhedged = pd.concat([UStodayfx, EUtodayfx, AUtoday1], axis=0, join='outer')
Globalgraphhedged = Globalhedged.transpose()
df_globalhedged = Globalgraphhedged.reset_index().rename(columns={'index': 'Maturity'})
df_globalhedged_long = df_globalhedged.melt(id_vars='Maturity', var_name='Curve', value_name='Yield')
fig_globalhedged = create_line_chart(df_globalhedged_long, 'Maturity', 'Yield', 'Curve', 
                                     f'Global Yield Curves (AUD Hedged) {date.today()}')
for trace in fig_globalhedged.data:
    trace.line.dash = 'solid'
Globalhedged_chart_html = pio.to_html(fig_globalhedged, full_html=False, include_plotlyjs='cdn')

10Y Yield Decomp (Nominal and Real)

In [9]:
import plotly.offline as pyo
import plotly.graph_objects as go 

yield_10y_decomp_list = ['GTII10 Govt','USGG10YR Index','USGGBE10 Index']
yield_10y_decomp = blp.bdh(tickers= yield_10y_decomp_list, flds=['px_last'], start_date='1-11-2014').droplevel(axis=1,level=1)
yield_10y_decomp.columns = ['US Real Yld 10Y','US Nominal Yld 10Y','US 10Y Breakeven Inflation']

# Create the figure
ten_10y_decomp = go.Figure()

# Add stacked area trace for "US Real Yld 10Y" (first, so it's at the bottom)
ten_10y_decomp.add_trace(go.Scatter(
    x=yield_10y_decomp.index,
    y=yield_10y_decomp['US 10Y Breakeven Inflation'],
    mode='lines',
    fill='tozeroy',
    name='10Y Breakeven',
    line=dict(color='#a0a197', width=0.5), #
    stackgroup='stack'  # Ensure proper stacking
))

# Add stacked area trace for "US 10Y Real Yield" (second, so it stacks on top of Breakeven)
ten_10y_decomp.add_trace(go.Scatter(
    x=yield_10y_decomp.index,
    y=yield_10y_decomp['US Real Yld 10Y'],
    mode='lines',
    fill='tonexty',
    name='10Y Real Yield',
    line=dict(color='#30415f', width=0.5),
    stackgroup='stack'  # Ensure proper stacking
))

# Add a line trace for "US Nominal Yld 10Y" (last, so it's a line on top)
ten_10y_decomp.add_trace(go.Scatter(
    x=yield_10y_decomp.index,
    y=yield_10y_decomp['US Nominal Yld 10Y'],
    mode='lines',
    name='10Y Nominal Yield',
    line=dict(color='#30415f', width=3),
))

# Update layout for styling
ten_10y_decomp.update_layout(
    width=1200,
    height=700,
    title=dict(
        text='US 10Y Yields: Real, Nominal and Breakeven Inflation',
        font=dict(size=24),  # Increase title size
        x=0.5,               # Center align title
        xanchor='center'
    ),
    legend=dict(
        orientation="h",     # Horizontal legend
        yanchor="bottom",
        y=-0.2,              # Position below chart
        xanchor="center",
        x=0.5,               # Center align legend
        font=dict(size=14),  # Increase legend font size
    ),
    xaxis_title="Date",
    yaxis_title="Yield (%)",
    template="plotly_white",
    font=dict(family="Montserrat"),
    yaxis=dict(rangemode="tozero"),
    hovermode="x unified",
)

ten_10y_decomp_html = pyo.plot(ten_10y_decomp, output_type='div', include_plotlyjs='cdn')

TIPS:

In [10]:
tips_list = ['GTII10 Govt','GTGBPII10Y Govt','GTAUDII10Y Govt']
tips = blp.bdh(tickers= tips_list, flds=['px_last'], start_date='1-08-2007', Per='W').droplevel(axis=1,level=1)
tips.columns = ['US','UK','AU']
tips_html = graph_performance(tips, "Inflation Linked Bonds")

Eco Surprises:

In [11]:
eco_surprise_list = ['GTII10 Govt','CESIEUR Index','CESIAUD Index','GTAUDII10Y Govt']
eco_surpris_df = blp.bdh(tickers= eco_surprise_list, flds=['px_last'], start_date='1-11-2021', Per='W').droplevel(axis=1,level=1)
eco_surpris_df.columns = ['US','EU','AU','China']
eco_surpris_df_html = graph_performance(eco_surpris_df, "Citi Economic Surprise Indices")

TS VALS:

In [82]:
valuation_metric_list_daily_aus =['PX_TO_BOOK_RATIO','LONG_TERM_PRICE_EARNINGS_RATIO', 'PX_TO_SALES_RATIO']
valuation_metric_list_daily =['PX_TO_BOOK_RATIO','LONG_TERM_PRICE_EARNINGS_RATIO','CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO']

sp500 = blp.bdh(tickers= 'SPX Index', flds = valuation_metric_list_daily, start_date = '1997-05-10', Per='W')
eur = blp.bdh(tickers= 'SXXP Index', flds = valuation_metric_list_daily, start_date = '1997-05-10', Per='W')
nky = blp.bdh(tickers= 'NKY Index', flds = valuation_metric_list_daily, start_date = '1997-05-10', Per='W')
asx = blp.bdh(tickers= 'AS51 Index', flds = valuation_metric_list_daily_aus, start_date = '1997-05-10', Per='W')
def winsorize_df(df, lower, upper):
    return df.apply(lambda x: np.clip(x, np.percentile(x.dropna(), lower * 100), np.percentile(x.dropna(), upper * 100)) if np.issubdtype(x.dtype, np.number) else x)

def process_index(raw_df, lower=0.01, upper=0.99):
    # Winsorize and calculate stats
    winsorized = winsorize_df(raw_df, lower, upper)
    
    # Calculate column-wise statistics
    medians = winsorized.median()
    stds = winsorized.std()
    
    # Normalize using column stats
    normalized = (winsorized - medians) / stds
    mean_series = normalized.mean(axis=1)
    
    # Create final DataFrame with bounds
    return pd.DataFrame({
        'z_score': mean_series,
        'mean': mean_series.mean(),  # Average of column medians
        'upper_bound': mean_series+1 ,
        'lower_bound': mean_series-1
    })

sp500 = process_index(sp500)
eur = process_index(eur) 
nky = process_index(nky)
asx = process_index(asx)

eur_daily = simp_graph_performance(eur, 'Europe Valuation Composite - Weekly - STOXX 600')
sp500_daily = simp_graph_performance(sp500, 'US Valuation Composite - Weekly - SP 500')
asx_daily = simp_graph_performance(asx, 'AUS Valuation Composite - Weekly - ASX 200')
nky_daily = simp_graph_performance(nky, 'Japan Valuation Composite - Weekly - Nikkei 225')

together_daily = pd.concat([nky, eur, sp500, asx], axis = 1, join='inner', keys = ['Nikkei','Stoxx 600','SP500','ASX 200'])
together_daily_ht = simp_graph_performance(together_daily.xs('z_score', axis=1,level=1), 'Valuation Composite - Weekly - Global')

In [ ]:
frequency = 'M'
start_date = '1997-05-10'
custom_colors = ['#003366', '#669bbc', '#f3a712', '#e4572e', '#87b1a1', '#a0a197', '#22a6a3', '#3454d1','#78adf9']

valuation_metric_list =['BEST_PE_RATIO','PX_TO_BOOK_RATIO','LONG_TERM_PRICE_EARNINGS_RATIO','CURRENT_EV_TO_T12M_EBITDA', 'PX_TO_SALES_RATIO']
valuation_list = ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']
regional_list = ['MXWO Index','SPX Index','SPW Index', 'NKY Index','AS51 Index','AS38 Index', 'MXEF Index','SXXP Index','MXKR Index','MXCN Index','UKX Index', 'MXWOU Index']
regionals_name = ['World','US','US Equal Weight', 'Japan','Australia','Aussie Smalls', 'EM','Europe','Korea','China','UK', 'World ex US']
factors_list = ['MXWO000V Index', 'M1WOEV Index', 'MXWO000G Index', 'M1WOQU Index','MXWOSC Index','MXWOLC Index','M1WOMVOL Index']
factors_names = ['Value', 'Enhanced Value', 'Growth', 'Quality', 'Small', 'Large', 'Low Volatility']

lower = 0.01
upper = 0.99
regional_vals = blp.bdh(tickers= regional_list, flds=valuation_metric_list, start_date= start_date, Per = frequency)
regional_vals_winsorized_add_composite = clean_data_after_bloomberg(regional_vals, regionals_name, valuation_list)
valuation_list = ['Forward PE', 'Price to Book', 'CAPE', 'EV/Trailing EBITDA', 'Price to Sales']
factor_vals = blp.bdh(tickers= factors_list, flds=valuation_metric_list, start_date= start_date, Per = frequency)
factor_vals_winsorized_add_composite = clean_data_after_bloomberg(factor_vals, factors_names, valuation_list)

generate_graphs(regional_vals_winsorized_add_composite, regionals_name, "reig_graphs")
generate_graphs(factor_vals_winsorized_add_composite, factors_names, "factor_graphs")

joined_reig_graphs_Forward_PE = "".join(reig_graphs_Forward_PE)
joined_reig_graphs_Price_to_Book = "".join(reig_graphs_Price_to_Book)
joined_reig_graphs_CAPE = "".join(reig_graphs_CAPE)
joined_reig_graphs_EV_Trailing_EBITDA = "".join(reig_graphs_EV_Trailing_EBITDA)
joined_reig_graphs_Price_to_Sales = "".join(reig_graphs_Price_to_Sales)
joined_reig_graphs_Valuation_Composite = "".join(reig_graphs_Valuation_Composite)

joined_factor_graphs_Forward_PE = "".join(factor_graphs_Forward_PE)
joined_factor_graphs_Price_to_Book = "".join(factor_graphs_Price_to_Book)
joined_factor_graphs_CAPE = "".join(factor_graphs_CAPE)
joined_factor_graphs_EV_Trailing_EBITDA = "".join(factor_graphs_EV_Trailing_EBITDA)
joined_factor_graphs_Price_to_Sales = "".join(factor_graphs_Price_to_Sales)
joined_factor_graphs_Valuation_Composite = "".join(factor_graphs_Valuation_Composite)

XS:

In [ ]:
########### TIME SERIES XS ###############
long = ['Value', 'Small', 'EM', 'World ex US', 'US Equal Weight']
short = ['Growth', 'Large', 'World', 'US', 'US']
cross_sectional_metric_to_compare = 'Valuation Composite'
cross_sectional_metric_to_compare_details = f'{cross_sectional_metric_to_compare} PB, CAPE, EV/EBITDA, P/S'

#Create Time Series
xs_df = pd.DataFrame({'Global Value / Global Growth':factor_vals_winsorized_add_composite[0][long[0]][cross_sectional_metric_to_compare]-factor_vals_winsorized_add_composite[0][short[0]][cross_sectional_metric_to_compare],
    'Global Small / Global Large':factor_vals_winsorized_add_composite[0][long[1]][cross_sectional_metric_to_compare]-factor_vals_winsorized_add_composite[0][short[1]][cross_sectional_metric_to_compare],
    'Emerging Markets / Developed Markets Spread': regional_vals_winsorized_add_composite[0][long[2]][cross_sectional_metric_to_compare]-regional_vals_winsorized_add_composite[0][short[2]][cross_sectional_metric_to_compare], 
    'International ex US Spread / US': regional_vals_winsorized_add_composite[0][long[3]][cross_sectional_metric_to_compare]-regional_vals_winsorized_add_composite[0][short[3]][cross_sectional_metric_to_compare],
    'S&P Equal Weight / S&P 500': regional_vals_winsorized_add_composite[0][long[4]][cross_sectional_metric_to_compare]-regional_vals_winsorized_add_composite[0][short[4]][cross_sectional_metric_to_compare]},
        index  = factor_vals_winsorized_add_composite[0].index)


cross_sectional_time_series_graph = simp_graph_performance(xs_df,'Cross-Sectional Valuation Composite Spreads')

# custom_colors = ['#003366', '#669bbc', '#f3a712', '#e4572e', '#87b1a1', '#a0a197', '#22a6a3', '#3454d1','#78adf9']
# cross_sectional_time_series_graph1 = px.line(data, width = 1500, height = 700, title = f'Cross-Sectional Valuation Composite Spreads', color_discrete_sequence=custom_colors)
# cross_sectional_time_series_graph1.update_layout(font=dict(family="Montserrat", size=13), title=dict(font=dict(family="Montserrat", size=22)), plot_bgcolor='white', legend=dict(
#         orientation="h",  # Horizontal orientation
#         yanchor="bottom",  # Anchor the legend to the bottom
#         y=-0.2,  # Position it below the plot area; adjust as necessary
#         xanchor="center",  # Center the legend horizontally
#         x=0.5  # Center the legend at the bottom
    # ))
# cross_sectional_time_series_graph1.update_xaxes(tickangle=45, title_text="", tickfont=dict(size=10))
# cross_sectional_time_series_graph1.update_yaxes(title_text=f'Valuation Composite (PB, EV/EBITDA, CAPE, P/S)')
# cross_sectional_time_series_graph1.update_yaxes(zeroline=True, zerolinewidth=0.5, zerolinecolor='lightgrey')  # Customize as needed

######## CURRENT XS ##########
factor_fig1 = cross_sectional_current_table_maker(factor_vals_winsorized_add_composite)
regional_fig1 = cross_sectional_current_table_maker(regional_vals_winsorized_add_composite)

# regional_fig = plot(regional_fig1, output_type='div', include_plotlyjs='cdn')
# factor_fig = plot(factor_fig1, output_type='div', include_plotlyjs='cdn')
# cross_sectional_time_series_graph = plot(cross_sectional_time_series_graph1, output_type='div', include_plotlyjs='cdn')

#xs_dict = {'Regional': regional_fig, 'Factor':factor_fig, 'XS Time-series Spreads': cross_sectional_time_series_graph}

Earnings Rev?:

In [17]:
from datetime import datetime, timedelta
today = datetime.today()
one_year_ago = today - timedelta(days=365)
one_year_ago.strftime("%Y-%m-%d")

#'PX_TO_CASH_FLOW','PX_TO_BOOK_RATIO','LONG_TERM_PRICE_EARNINGS_RATIO','CURRENT_EV_TO_T12M_EBITDA'
metric1 = 'INDX_WEIGHTED_EST_ERN'
metric3 = 't12_eps_aggte'
metric2 = 'tot_return_index_gross_dvds'
regional_list = ['MXWO Index','SPX Index','SPW Index', 'NKY Index','AS51 Index','MXEF Index','SXXP Index','MXKR Index','MXCN Index','UKX Index']
regionals_name = ['World','US','US_Equal_Weight','Japan','Australia','Emerging Markets','Europe','Korea','China','UK']
regional_earnings_fwd = blp.bdh(tickers= regional_list, flds=[metric1], start_date=one_year_ago.strftime("%Y-%m-%d"), BEST_FPERIOD_OVERRIDE = 'BF').droplevel(axis=1,level=1)
regional_realized_earnings = blp.bdh(tickers= regional_list, flds=[metric3], start_date=one_year_ago.strftime("%Y-%m-%d")).droplevel(axis=1,level=1)
regional_earnings_fwd.columns = regionals_name
regional_realized_earnings.columns = regionals_name

earnings_revisions_table = (((regional_earnings_fwd / regional_realized_earnings)-1)*100).round(2)
blend4q_earnings_rev = simp_graph_performance(earnings_revisions_table, 'Blended 4Q Forward Earnings Revisions (%)')

DMW:

In [ ]:
def off_returns_dmw(series, days_ago):
    today_date = series.iloc[-2].name
    days_ago = days_ago + 1
    
    # Check if today_date is a Monday (weekday == 0)
    if today_date.weekday() == 0:  # Monday
        a1 = today_date - pd.DateOffset(days_ago + 2)
    else:
        a1 = today_date - pd.DateOffset(days_ago)
    
    prev_workday = a1 - pd.offsets.BusinessDay(1)
    final_workday = a1 + pd.offsets.BusinessDay(1)

    if days_ago > 365:
        final_number = (series.loc[today_date] / series.loc[final_workday]) ** (1 / (days_ago / 365)) - 1
    else:
        final_number = (series.loc[today_date] / series.loc[final_workday]) - 1

    return final_number

asset = pd.read_csv('temp_shortened.csv',index_col=0).drop_duplicates()
list_of_assets = asset.index
dmw_raw = blp.bdh(tickers = list_of_assets.to_list(), flds= 'tot_return_index_gross_dvds', start_date= '31-01-2010').droplevel(axis=1,level=1)
dmw_raw.index = pd.to_datetime(dmw_raw.index)
names_dmw  = blp.bdp(list_of_assets.to_list(), 'long_comp_name')
types  = blp.bdp(list_of_assets.to_list(), 'SECURITY_TYP')

df_returns = pd.DataFrame({
    "1d": off_returns_dmw(dmw_raw, 1),
    "3d": off_returns_dmw(dmw_raw, 3),
    "1w": off_returns_dmw(dmw_raw, 7),
    "30d": off_returns_dmw(dmw_raw, 30),
    "3m": off_returns_dmw(dmw_raw, 90),
    "6m": off_returns_dmw(dmw_raw, 182),
    "1y": off_returns_dmw(dmw_raw, 365),
    "3y": off_returns_dmw(dmw_raw, 365*3),
    "5y": off_returns_dmw(dmw_raw, 365*5),
    "7y": off_returns_dmw(dmw_raw, 365*7),
    "10y": off_returns_dmw(dmw_raw, 365*10),
})
df_returns = df_returns  # Exclude the last few non-numeric columns if necessary
df_returns = df_returns.round(4)
df_returns['Name'] = names_dmw
df_returns['Asset Class'] = types
df_returns = df_returns.set_index('Name')
df_returns2 = df_returns.copy()
daily_market_watch_table = df_returns.to_html(classes="table table-striped table-hover table-bordered")

equity_asset_dmw = (df_returns[df_returns['Asset Class']=='Equity Index']).drop('Asset Class',axis=1)
debt_asset_dmw = (df_returns[df_returns['Asset Class']=='Fixed Income Index']).drop('Asset Class',axis=1)
other_asset_dmw = (df_returns[
    (df_returns['Asset Class'] == 'Index') | (df_returns['Asset Class'] == 'Commodity Index')
]).drop('Asset Class',axis=1)

equity_styled = equity_asset_dmw.to_html(escape=False, classes='table table-striped table-bordered')
debt_styled = debt_asset_dmw.to_html(escape=False, classes='table table-striped table-bordered')
other_styled = other_asset_dmw.to_html(escape=False, classes='table table-striped table-bordered')

Stock / Bond Correlation:

In [19]:
corr = dmw_raw[['luattruu Index','spx index']].dropna()
corr1 = (corr/corr.shift())-1
rolling_corr = corr1['luattruu Index'].rolling(window=90).corr(corr1['spx index'])
rolling_corr = rolling_corr.dropna()
graph_corr_stockbond = graph_performance(rolling_corr, "90 Day Correlation (MSCI ACWI AND GLOBAL AGG)")

In [ ]:
def calc_return_1m_to_10y(sector_df, sector_names, cmap="RdYlGn"):
    sector_df.columns = sector_names
    # Calculate returns
    onemonth = (sector_df.iloc[-1] / sector_df.iloc[-2]) - 1
    threemonth = (sector_df.iloc[-1] / sector_df.iloc[-4]) - 1
    sixmonth = (sector_df.iloc[-1] / sector_df.iloc[-7]) - 1
    oneyr = (sector_df.iloc[-1] / sector_df.iloc[-13]) - 1
    threeyr = ((sector_df.iloc[-1] / sector_df.iloc[-37]) ** (1/3)) - 1
    fiveyr = ((sector_df.iloc[-1] / sector_df.iloc[-61]) ** (1/5)) - 1
    tenyr = ((sector_df.iloc[-1] / sector_df.iloc[-121]) ** (1/10)) - 1

    # Create a dictionary of returns
    returns_dict = {
        '1 Month': onemonth,
        '3 Months': threemonth,
        '6 Months': sixmonth,
        '1 Year': oneyr,
        '3 Years': threeyr,
        '5 Years': fiveyr,
        '10 Years': tenyr
    }

    # Create a DataFrame from the dictionary
    returns_df = pd.DataFrame(returns_dict, index=sector_df.columns).applymap(lambda x: f"{x*100:.2f}%")

    return returns_df

#regions
dmw_daily_rets = (dmw_raw/dmw_raw.shift())-1
returns_2025 = dmw_daily_rets[dmw_daily_rets.index.year == 2025]
rebase_ytd = (1+returns_2025).cumprod()
major_equity_ytd = rebase_ytd[['AS51 INDEX','spx index','nky index','mxeu index','mxef index','UKX index','shcomp index','KOSPI Index','NDX Index','spw index','mxwd index']].ffill()
major_equity_ytd.columns = ['ASX200','S&P 500','Nikkei 225','MSCI Europe','MSCI EM','FTSE 100','Shanghai Composite','KOSPI Index','Nasdaq Comp','S&P 500 Equal Weight','MSCI ACWI']
major_equity_ytd = major_equity_ytd.fillna(1)
graph_for_regional_equity_ytd = simp_graph_performance(major_equity_ytd, "Major Equity YTD Performance (Local Currency)")

#factors
factors_world = ['MXWO000V Index','MXWO000G Index','MXWOSC Index','MXWOLC Index','M1WOMOM Index','M1WOQU Index','M1WOMVOL Index','NU748615 Index','M1WOEV Index']
factors_perf = blp.bdh(tickers = factors_world, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2023', FX='USD').droplevel(axis=1,level=1)
factors_perf.index = pd.to_datetime(factors_perf.index)
factors_perf.columns = ['Value','Growth','Small','Large','Momentum','Quality','Min Vol','Quality Smalls','Enhanced Value']
returns_factors_major = (factors_perf / factors_perf.shift()) - 1
returns_2025_factors = returns_factors_major[returns_factors_major.index.year == 2025]
cumulative_factor = (1+returns_factors_major).cumprod()
cumulative_factor_2025 = (1+returns_2025_factors).cumprod()
graph_for_factor_equity = simp_graph_performance(cumulative_factor.dropna(), "Global Factor Long only Performance")
graph_for_factor_equity_ytd = graph_performance(cumulative_factor_2025, "Global Factor Long only YTD Performance")


us1 = ['s5matr Index', 's5enrs index', 's5finl index', 's5hlth index', 's5cond index', 's5cons index', 's5inft index', 's5rlst index', 's5util index', 's5tels index', 's5indu index']
au1 = ['AS51MATL Index', 'as51engy index', 'as51fin index', 'as51hc index', 'as51cond index', 'as51cons index', 'as51it index', 'as51prop index', 'as51util index', 'as51tele index', 'as51indu index']
jap1 = ['MXJP0MT Index', 'MXJP0EN Index', 'MXJP0FN Index', 'MXJP0HC Index', 'MXJP0CD Index', 'MXJP0CS Index', 'MXJP0IT Index', 'MXJP0RL Index', 'MXJP0UT Index', 'MXJP0TC Index', 'MXJP0IN Index']
uk1 = ['MXGB0MT Index', 'MXGB0EN Index', 'MXGB0FN Index', 'MXGB0HC Index', 'MXGB0CD Index', 'MXGB0CS Index', 'MXGB0IT Index', 'MXGB0RL Index', 'MXGB0UT Index', 'MXGB0TC Index', 'MXGB0IN Index']
eu1 = ['MXEU0MT Index', 'MXEU0EN Index', 'MXEU0FN Index', 'MXEU0HC Index', 'MXEU0CD Index', 'MXEU0CS Index', 'MXEU0IT Index', 'MXEU0RE Index', 'MXEU0UT Index', 'MXEU0TC Index', 'MXEU0IN Index']
#korea1 = ['MXKR0MT Index', 'MXKR0UT Index', 'MXKR0EN Index', 'MXKR0MT Index', 'MXKR0CD Index', 'MXKR0CS Index', 'MXKR0IT Index', 'MXKR0FN Index', 'MXKR0HC Index', 'MXKR0RL Index', 'MXKR0IN Index']

us_sector = blp.bdh(tickers = us1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M').droplevel(axis=1,level=1)
au_sector = blp.bdh(tickers = au1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M').droplevel(axis=1,level=1)
jap_sector = blp.bdh(tickers = jap1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M').droplevel(axis=1,level=1)
uk_sector = blp.bdh(tickers = uk1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M').droplevel(axis=1,level=1)
eu_sector = blp.bdh(tickers = eu1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M').droplevel(axis=1,level=1)
sector_names = ['Mat', 'En', 'Fin', 'HC', 'CS', 'CD', 'IT', 'Real Est', 'Util', 'Comm', 'Indus']
#kor_sector = blp.bdh(tickers = korea1, flds= 'tot_return_index_gross_dvds', start_date= '31-07-2014', Per='M')

us_sector1 = calc_return_1m_to_10y(us_sector, sector_names).to_html(escape=False, classes='table table-striped table-bordered')
au_sector1 = calc_return_1m_to_10y(au_sector, sector_names).to_html(escape=False, classes='table table-striped table-bordered')
jap_sector1 = calc_return_1m_to_10y(jap_sector, sector_names).to_html(escape=False, classes='table table-striped table-bordered')
uk_sector1 = calc_return_1m_to_10y(uk_sector, sector_names).to_html(escape=False, classes='table table-striped table-bordered')
eu_sector1 = calc_return_1m_to_10y(eu_sector, sector_names).to_html(escape=False, classes='table table-striped table-bordered')
# kor_sector1 = calc_return_1m_to_10y(kor_sector, sector_names).to_html()

Aussie-Section:

In [ ]:
#VALUATIONS
frequency = 'M'
start_date = '2003-05-10'
lower = 0.01
upper = 0.99
custom_colors = ['#003366', '#669bbc', '#f3a712', '#e4572e', '#87b1a1', '#a0a197', '#22a6a3', '#3454d1','#78adf9']

avaluation_metric_list =['BEST_PE_RATIO','PX_TO_BOOK_RATIO', 'LONG_TERM_PRICE_EARNINGS_RATIO','INDX_PX_SALES']
avaluation_list = ['Forward PE', 'Price to Book', 'CAPE', 'Price to Sales']
aregional_list =  ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS51 Index']
aregionals_name = ['Banks','Resources','Equal Weight','Materials','ASX 200']

aregional_vals = blp.bdh(tickers= aregional_list, flds=avaluation_metric_list, start_date= start_date, Per = frequency)
aregional_vals_winsorized_add_composite = clean_data_after_bloomberg(aregional_vals, aregionals_name, avaluation_list)

generate_graphs(aregional_vals_winsorized_add_composite, aregionals_name, "aus_reig_graphs")
aus_joined_reig_graphs_Forward_PE = "".join(aus_reig_graphs_Forward_PE)
aus_joined_reig_graphs_Price_to_Book = "".join(aus_reig_graphs_Price_to_Book)
aus_joined_reig_graphs_CAPE = "".join(aus_reig_graphs_CAPE)
aus_joined_reig_graphs_Price_to_Sales = "".join(aus_reig_graphs_Price_to_Sales)
aus_joined_reig_graphs_Valuation_Composite = "".join(aus_reig_graphs_Valuation_Composite)


#EARNINGS?
metric1 = 'INDX_WEIGHTED_EST_ERN'
metric3 = 't12_eps_aggte'
metric2 = 'tot_return_index_gross_dvds'
aregional_list = ['AS51BANX Index','AS45 Index','MVMVWTRG Index','AS51MATL Index','AS52 Index']
aregionals_name = ['Banks','Resources','Equal Weight','Materials','ASX 200']
aregional_earnings_fwd = blp.bdh(tickers= aregional_list, flds=[metric1], start_date=one_year_ago.strftime("%Y-%m-%d"), BEST_FPERIOD_OVERRIDE = 'BF').droplevel(axis=1,level=1)
aregional_realized_earnings = blp.bdh(tickers= aregional_list, flds=[metric3], start_date=one_year_ago.strftime("%Y-%m-%d")).droplevel(axis=1,level=1)
aregional_earnings_fwd.columns = aregionals_name
aregional_realized_earnings.columns = aregionals_name

aearnings_revisions_table = (((aregional_earnings_fwd / aregional_realized_earnings)-1)*100).round(2)
aussie_blend4q_earnings_rev = simp_graph_performance(aearnings_revisions_table, 'Blended 4Q Forward Earnings Revisions (%)')

### Base Case Graphs:

In [23]:
list_of_base_case_graphs = ['CESIUSD Index','CESIEUR Index','CESIAUD Index','CESICNY Index']
eco_surpris_df = blp.bdh(tickers= list_of_base_case_graphs, flds=['px_last'], start_date='1-11-2021', Per='M').droplevel(axis=1,level=1)
eco_surpris_df.columns = ['US','EU','AU','China']
eco_surpris_df_html = graph_performance(eco_surpris_df, "Citi Economic Surprise Indices")

Macro:

In [24]:
# GDP FORECAST 2025
names = ['USA','AUS','GBR','EUR','CHN','JAP','KOR','BRA','IND']
gdp_consensus_list = ['ECGDUS 25 Index','ECGDAU 25 Index','ECGDGB 25 Index','ECGDEU 25 Index','ECGDCN 25 Index','ECGDJP 25 Index','ECGDKR 25 Index','ECGDBR 25 Index','ECGDIN 25 Index']
gdp_consensus_data = blp.bdh(tickers= gdp_consensus_list, flds=['px_last'], start_date='2023-9-1', Per = 'W').droplevel(axis=1,level=1)
gdp_consensus_data.columns = names
gdp_consensus_html = graph_performance(gdp_consensus_data, "Bloomberg Consensus Real GDP Forecast YoY 2025")

In [25]:
#MANUFACT PMI
manufacturing_PMI = ['NAPMPMI Index','MPMIAUMA Index','MPMIGBMA Index','MPMIEZMA Index','CPMINDX Index','MPMIJPMA Index','MPMIKRMA Index','MPMIBRMA Index','MPMIINMA Index']
names = ['USA','AUS','GBR','EUR','CHN','JAP','KOR','BRA','IND']

cash_label = ['USA','AUS','GBR','ECB','JAP','CHE','HKG', 'CAN', 'KOR','BRA','IND','MEX', 'IDN','TWN']
cash_bbg = ['FEDL01 Index',	'RBACOR Index'	,'UKBRBASE Index'	,'EUR003M Index',	'JY0003M Index',	'BISPMSWI Index',	'BISPMCHI Index', 'CABROVER Index', 'KORP7DR Index',
            'BZSTSETA Index', 'INRPYLDP Index','MXONBR Index', 'IDBIRRPO Index','TAREDSC Index']

df = blp.bdh(tickers= manufacturing_PMI, flds=['px_last'], start_date='2020-10-10', Per = 'M')
df.index = pd.to_datetime(df.index)
cleaned_df = (df).droplevel(level=1,axis=1).dropna()
cleaned_df.columns = names
graph_df_pmi = pd.concat([cleaned_df.iloc[-1], cleaned_df.iloc[-12]], axis=1)
graph_df_pmi.columns = ['Current','1 Year Ago']

manufac1 = px.bar(graph_df_pmi, x=graph_df_pmi.index, y=['Current', '1 Year Ago'],
             title="Manufacturing PMI: Current vs 1 Year Ago",
             labels={"value": "PMI Index", "index": ""},
             barmode='group',
             color_discrete_sequence=['#30415f', '#669bbc'])

# Updating layout for consistency with rate futures chart
manufac1.update_layout(
    font_family="Montserrat",
    title={"font": {"size": 22}},
    yaxis=dict(title="PMI Index", titlefont=dict(color="black"), tickfont=dict(color="black"), gridcolor="lightgray"),
    plot_bgcolor="white",
    paper_bgcolor="white",
    width=950,
    height=600,
    legend=dict(orientation="h", y=-0.075, x=0.5, xanchor="center")
)

manufac = plot(manufac1, output_type='div', include_plotlyjs='cdn')

In [26]:
lei = blp.bdh(tickers= 'LEI YOY Index', flds=['px_last'], start_date='1960-1-1', Per = 'M')
lei_z = (lei-lei.median())/(lei.std())
lei_z_table = graph_performance(lei_z.droplevel(axis=1,level=1), "US Leading Economic Indicator YoY")

In [27]:
gdp_global = ['GDP CYOY Index', 'AUNAGDPY Index','UKGRABIY Index', 'EUGNEMUY Index', 'CNGDPYOY Index','JGDPNSAQ Index', 'KOGDPYOY Index', 'BZGDYOY% Index', 'IGQREGDY Index']
df1 = blp.bdh(tickers= gdp_global, flds=['px_last'], start_date='2020-10-10', Per = 'Q')
df1.index = pd.to_datetime(df1.index)
cleaned_df1 = (df1).droplevel(level=1,axis=1).dropna()
real_gdp = pd.concat([cleaned_df1.iloc[-1], cleaned_df1.iloc[-5]],axis=1)
real_gdp.index = names
real_gdp_table = real_gdp.to_html(classes="table table-striped table-hover table-bordered")

styled_table1 = (
    real_gdp.style
    .format(precision=2)
    .set_table_styles([
        {'selector': 'td:hover', 'props': [('background-color', '#30415f')]},
        {'selector': 'th:not(.index_name)', 'props': [('background-color', '#30415f'), ('color', 'white'), ('text-align', 'center')]},
        {'selector': 'td', 'props': [('font-size', '14px'), ('text-align', 'center'), ('width', '80px'), ('border', '1px solid #ddd')]},
        {'selector': 'th', 'props': [('text-align', 'left'), ('border', '1px solid #ddd'), ('width', '80px')]},
        {'selector': 'table', 'props': [('border-collapse', 'collapse')]}
    ])
    .to_html(table_attrs={'id': 'marketWatchTable', 'class': 'display'})
)

real_gdp_table = styled_table1

In [28]:
drivers_of_global_eps = ['CHEFTYOY Index','KOEXTOTY Index','SIASTOTL Index','SIEXEYOY Index','LMCADS03 Comdty']

Macro Regime:

In [29]:
variables_for_recession_probability = ['LEI YOY Index',	'NFCIINDX Index', 'OEUSZREX Index',	'USGG2YR Index', 'FEDL01 Index', 'NAPMNEWO Index', 'NHSPSTOT Index']
recession_probably_raw_data = blp.bdh(tickers= variables_for_recession_probability, flds=['px_last'], start_date='1960-1-1', Per = 'M').droplevel(axis=1,level=1)
recession_probably_z_scores = (recession_probably_raw_data-recession_probably_raw_data.mean())/recession_probably_raw_data.std()
#average_z_score = recession_probably_z_scores.mean(axis=1)

## Technicals:

In [30]:
def plot_moving_averages(dataframe):
    graph_html = {}
    ma_specs = {
        '50d_MA': {'color': 'silver', 'dash': 'dot'},
        '200d_MA': {'color': 'steelblue', 'dash': 'dot'}
    }
    for index_name in dataframe.columns:
        temp_df = dataframe[[index_name]].copy()
        temp_df['50d_MA'] = temp_df[index_name].rolling(window=50).mean()
        temp_df['200d_MA'] = temp_df[index_name].rolling(window=200).mean()

        fig = go.Figure()
        # Main price line
        fig.add_trace(go.Scatter(
            x=temp_df.index,
            y=temp_df[index_name],
            mode='lines',
            name=f'{index_name} Price',
            line=dict(color='#30415f', width=2)
        ))
        # Moving averages
        for ma_type, style in ma_specs.items():
            fig.add_trace(go.Scatter(
                x=temp_df.index,
                y=temp_df[ma_type],
                mode='lines',
                name=ma_type,
                line=dict(
                    color=style['color'],
                    dash=style['dash'],
                    width=2
                )
            ))

        # Update layout with larger size
        fig.update_layout(
            title=f'{index_name} Moving Averages',
            xaxis_title='Date',
            yaxis_title='Price',
            template='plotly_white',
            hovermode='x unified',
            font=dict(family="Montserrat, sans-serif"),  # Set Montserrat as the font
            title_font=dict(family="Montserrat, sans-serif"),  # Ensure title is also Montserrat
            legend_font=dict(family="Montserrat, sans-serif"),  # Set legend font to Montserrat
            width=1400,  # Adjust width (increase as needed)
            height=700   # Adjust height (increase as needed)
        )

        # Update axis label fonts
        fig.update_xaxes(title_font=dict(family="Montserrat, sans-serif"))
        fig.update_yaxes(title_font=dict(family="Montserrat, sans-serif"))

        graph_html[index_name] = fig.to_html(
            full_html=False,
            include_plotlyjs='cdn'
        )
    
    return graph_html

In [31]:
major_index_tickers = ['AS51 INDEX','spx index','nky index','mxeu index','mxef index','UKX index','shcomp index','KOSPI Index','NDX Index','spw index','mxwd index']
major_index_names = ['ASX200','S&P 500','Nikkei 225','MSCI Europe','MSCI EM','FTSE 100','Shanghai Composite','KOSPI Index','Nasdaq Comp','S&P 500 Equal Weight','MSCI ACWI']
technicals_major_index_raw_data = blp.bdh(tickers= major_index_tickers, flds=['px_last'], start_date='2005-6-1').droplevel(axis=1,level=1)
technicals_major_index_raw_data.columns = major_index_names
technicals_major_index_raw_data = technicals_major_index_raw_data.ffill()
technicals_graphs_html = plot_moving_averages(technicals_major_index_raw_data)

Static Images & Misc before HTML:

In [32]:
Our_Asset_Class_Views = '<img src="https://i.imgur.com/wMquJZo.png" width="800">'
Fla = '<img src="https://i.imgur.com/xPi0AgA.png" width="800">'
Funda = '<img src="https://i.imgur.com/fYmUJ4d.png" width="800">'
Trad = '<img src="https://i.imgur.com/5hqqFJP.png" width="800">'
Cfs_fc = '<img src="https://i.imgur.com/Nh18Zhx.png" width="800">'
basecase = '<img src="https://i.imgur.com/mWuqeJA.png" width="700">'
cape_chart = '<img src="https://i.imgur.com/RsszGsQ.png" width="700">'
concentration = '<img src="https://i.imgur.com/JJQxKdV.png" width="700">'
eco_surprise = '<img src="https://i.imgur.com/mdfxHrD.png" width="700">'
recession = '<img src="https://i.imgur.com/6zODN1y.png" width="700">'
sys_outputs = '<img src="https://i.imgur.com/ENC01In.png" width="700">'
region_positioning = '<img src="https://i.imgur.com/VFEfrIh.png" width="700">'
returns_2022 = '<img src="https://i.imgur.com/e6xcaLa.png" width="700">' 
crsp_dimensional = '<img src="https://i.imgur.com/Rz9Kqrz.png" width="800">'
deficit_matrix = '<img src="https://imgur.com/0e160541-a5fc-4e44-9543-dd2453b4f153" width="800">'
tariff_2025 = '<img src="https://i.imgur.com/ZKUHcjO.png" width="800">'
tariff_2025_asia = '<img src="https://i.imgur.com/T9y1j74.png" width="800">'

# Scrape from Nowcast
today_date = datetime.now().strftime("%B %d, %Y")
url = "https://www.atlantafed.org/cqer/research/gdpnow"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")

page_text = soup.get_text()
keyword = "Latest estimate:"
pos = page_text.find(keyword)

if pos != -1:
    start_index = pos + len(keyword)
    # Safely get the next 24 characters (or fewer if the string is short)
    next_24_chars = page_text[start_index : start_index + 24]
    print("Latest estimate:", next_24_chars)
else:
    print("No 'Latest estimate:' found in the text.")

Latest estimate:  -2.8 percent — April 03


OG OG:

In [ ]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Innova Asset Mangagement Internal Dashboard</title>
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700&display=swap" rel="stylesheet">
  <!-- Bootstrap CSS -->
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    /* Base styles */
    body {{
      margin: 0;
      font-family: 'Montserrat', sans-serif;
    }}
    /* Header Section */
    .site-header {{
      width: 100%;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}
    .palette {{
      display: flex;
    }}
    .palette .color-block {{
      flex: 1;
      height: 8px;
    }}
    .header-title-container {{
      background-color: #30415f;
      padding: 10px;
      text-align: center;
      color: #fff;
      position: relative;
    }}
    /* Home button styling */
    .home-button {{
      position: absolute;
      right: 20px;
      top: 10px;
    }}
    /* Main container for sidebar + content */
    .main-container {{
      display: flex;
      height: calc(100vh - 60px); /* Adjust based on header height */
    }}
    .sidebar {{
      width: 250px;
      background-color: #f5f5f5;
      padding: 10px;
      border-right: 1px solid #ddd;
      overflow-y: auto;
    }}
    .content {{
      flex-grow: 1;
      padding: 20px;
      overflow-y: auto;
    }}
    /* First-level tabs styling */
    #sidebar-menu > a.list-group-item {{
      background-color: #30415f;
      color: #fff;
      font-weight: bold;
      border: none;
    }}
    #sidebar-menu > a.list-group-item:hover {{
      background-color: #30415f; 
    }}
    /* Sub-tabs remain default or can be customized separately */
    .list-group-item {{
      cursor: pointer;
    }}
    /* Image styling */
    .content-pane img {{
      display: block;
      margin: 20px auto; /* Adds spacing between images */
      max-width: 100%; /* Ensures images don’t overflow */
    }}

    /* Graph styling: center divs, canvases, iframes, and svgs inside .content-pane */
    .content-pane div,
    .content-pane canvas,
    .content-pane iframe,
    .content-pane svg {{
      display: block;
      margin: 20px auto; /* Centers the element */
      max-width: 100%;   /* Prevents overflow */
    }}

  </style>
</head>
<body>
  <!-- Header Section with Palette and Title -->
  <header class="site-header">
    <div class="palette">
      <div class="color-block" style="background-color: #30415f;"></div>
      <div class="color-block" style="background-color: #30415f;"></div>
    </div>
    <div class="header-title-container">
      <h1>Innova Asset Mangagement Internal Dashboard</h1>
      <a href="#" class="btn btn-light home-button" onclick="showContent('landing')">Home</a>
    </div>
  </header>

  <!-- Main Container: Sidebar + Content -->
  <div class="main-container">
    <!-- Sidebar with Hierarchical Tabs -->
    <div class="sidebar">
      <div class="list-group" id="sidebar-menu">
        <!-- Broad Category 1: Innova SMA Analytics -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseDMWW" aria-expanded="false" aria-controls="collapseDMWW">
          Innova SMA Analytics
        </a>
        <div class="collapse" id="collapseDMWW" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmww1')">Asset Class Views</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmww2')">Systematic Outputs</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmww3')">Target Spreadsheets</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmww4')">Evergreen Innova Charts 2025</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmww5')">Positioning</a>
        </div>

        <!-- Broad Category 2: Daily Market Watch -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseDMW" aria-expanded="false" aria-controls="collapseDMW">
          Daily Market Watch
        </a>
        <div class="collapse" id="collapseDMW" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmw1')">Table Format</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmw2')">Graph Format</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmw3')">Factor Performance</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmw4')">Sector Performance</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('dmw5')">Daily Everyday Movements</a>
        </div>
        
        <!-- Broad Category 3: Region Equity Valuations -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseVF" aria-expanded="false" aria-controls="collapseVF">
          Region Equity Valuations
        </a>
        <div class="collapse" id="collapseVF" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf1')">Daily Valuations on Popular Benchmarks</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf8')">Adjustments to Valuations</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf0')">Forward PE</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf9')">CAPE Ratio (Bloomberg Calc)</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf2')">Price to Book</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf3')">Price to 5Y Sales</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf4')">EV to EBITDA</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf5')">Composite Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf6')">Cross-Sectional Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vf7')">REITs Valuation</a>
        </div>

        <!-- Broad Category 4: Factor Equity Valuations -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseVAF" aria-expanded="false" aria-controls="collapseVAF">
          Factor Equity Valuations
        </a>
        <div class="collapse" id="collapseVAF" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf1')">Global Factor Matrix</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf8')">Forward PE</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf9')">CAPE Ratio (Bloomberg Calc)</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf2')">Price to Book</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf3')">Price to 5Y Sales</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf4')">EV to EBITDA</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf5')">Composite Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vaf6')">Cross-Sectional Valuation</a>
        </div>

        <!-- Broad Category 5: Earnings & Fundamentals -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseVVF" aria-expanded="false" aria-controls="collapseVVF">
          Earnings & Fundamentals
        </a>
        <div class="collapse" id="collapseVVF" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf1')">Reigonal Earnings Growth / Revisions</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf9')">Magnificent 7 Earnings & Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf2')">Factor Earnings Growth / Revisions</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf3')">Sources of Return & Box Valuations</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf4')">Key Company Earnings</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf5')">S&P Profit Margins</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf6')">Global Sector Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf7')">Australia Sector Valuation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('vvf8')">Anaylst Estimate... list?</a>
        </div>
        
        <!-- Broad Category 6: Macroeconomic -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseMacro" aria-expanded="false" aria-controls="collapseMacro">
          Macroeconomic
        </a>
        <div class="collapse" id="collapseMacro" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro1')">Macroeconomic Regime</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro2')">US Growth</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro3')">US Inflation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro4')">US Labour Market</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro5')">US Recession Probability</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro6')">US Stock Bond Correlation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro7')">Domestic Growth</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro8')">Domestic Inflation</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro9')">Domestic Labour Market</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro10')">Global Economic Matrix (Growth/Inf)</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('macro11')">Global Liquidity</a>
        </div>
        
        <!-- Broad Category 7: Technicals / Sentiment / Risk Aversion -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseTech" aria-expanded="false" aria-controls="collapseTech">
          Technicals / Sentiment / Risk Aversion
        </a>
        <div class="collapse" id="collapseTech" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('tech1')">Moving Day Average Graphs</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('tech2')">Table of Index Technicals</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('tech3')">Volatility of Asset Classes</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('tech4')">Net Flows</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('tech5')">Fear / Greed Type Indicators</a>
        </div>
        
        <!-- Broad Category 8: Bond Futures and Yield Curve -->
        <a class="list-group-item list-group-item-action" data-toggle="collapse" href="#collapseMisc" aria-expanded="false" aria-controls="collapseMisc">
          Bond Futures and Yield Curve 
        </a>
        <div class="collapse" id="collapseMisc" data-parent="#sidebar-menu">
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('misc1')">Rate Cut/Hike Pricing</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('misc2')">Yield Curves</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('misc3')">AU Credit Spreads</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('misc4')">Global Credit Spreads</a>
          <a class="list-group-item list-group-item-action pl-4" onclick="showContent('misc5')">Currency Valuations PPP</a>
        </div>
      </div>
    </div>
    
    <!-- Content Area for Charts -->
    <div class="content" id="content-area">
      <!-- Landing Page -->
      <div id="landing" class="content-pane">
        <h2>Welcome Innova Team</h2>
        <p>Please use the sidebar to navigate the different sections. Below are some relevant weekly updates as of the {today_date} in macro/markets:<:</p>
        <ul>
          <li><a href="https://www.cnbc.com/finance/" target="_blank">CNBC Finance Top Headlines for {today_date}</a></li>
          <li> <a href="https://tradingeconomics.com/calendar" target="_blank">Economic Calender/Releases as of {today_date}</a></li>
          <br>
          <br>
          <h2>Top News of the Week:</h2>
          <li>Donald Trump and Vladimir Putin will speak Tuesday as the US pushes for a Ukraine ceasefire. Ahead of the talks, which Trump said will focus on territory, Putin issued a decree allowing some American and other Western funds to sell Russian securities. </li>
          <li>BYD unveiled a battery and charging system that was capable of providing 470 kilometers of range in five minutes in tests on its new Han L sedan. US auto giants have feared a world of Chinese dominance for decades.</li>
          <li>Intel shares rose ~30% after Reuters reported incoming CEO Lip-Bu Tan has considered significant changes to its chip manufacturing methods and AI strategies..</li>
          <li><a href="https://www.atlantafed.org/cqer/research/gdpnow" target="_blank">Latest GDP Nowcast is {next_24_chars}</a></li>
          <li>{crsp_dimensional}</li>
        </ul>
      </div>
      
      <!-- Innova SMA Analytics -->
      <div id="dmww1" class="content-pane" style="display:none;">
        <div>{Our_Asset_Class_Views}</div>
      </div>
      <div id="dmww2" class="content-pane" style="display:none;">
        <div>{sys_outputs}</div>
        <div>{recession}</div>
      </div>
      <div id="dmww3" class="content-pane" style="display:none;">
        <div>{Funda}</div>
        <div>{Fla}</div>
        <div>{Trad}</div>
        <div>{Cfs_fc}</div>
      </div>
      <div id="dmww4" class="content-pane" style="display:none;">
        <div>{basecase}</div>
        <div>{eco_surprise}</div>
        <div>{concentration}</div>
        <div>{cape_chart}</div>
        <div>{returns_2022}</div>
      </div>
      <div id="dmww5" class="content-pane" style="display:none;">
        {region_positioning}
      </div>

      <!-- Daily Market Watch (Broad Category 1) -->
      <div id="dmw1" class="content-pane" style="display:none;">
        <h2>{'Equities only'}</h2>
        <div>{equity_styled}</div>
        <h2>{'Fixed Income only'}</h2>
        <div>{debt_styled}</div>
        <h2>{'Other'}</h2>
        <div>{other_styled}</div>
      </div>
      <div id="dmw2" class="content-pane" style="display:none;">
        {graph_for_regional_equity_ytd}
      </div>
      <div id="dmw3" class="content-pane" style="display:none;">
        <div>{graph_for_factor_equity_ytd}</div>
        <div>{graph_for_factor_equity}</div>
      </div>
      <div id="dmw4" class="content-pane" style="display:none;">
        <h2>{'US Sector'}</h2>
        <div>{us_sector1}</div>
        <h2>{'AU Sector'}</h2>
        <div>{au_sector1}</div>
        <h2>{'Japan Sector'}</h2>
        <div>{jap_sector1}</div>
        <h2>{'EU Sector'}</h2>
        <div>{eu_sector1}</div>
        <h2>{'UK Sector'}</h2>
        <div>{uk_sector1}</div>
      </div>
      <div id="dmw5" class="content-pane" style="display:none;">
        {ten_10y_decomp_html}
      </div>
      
      <!-- Region Equity Valuations (Broad Category 2) -->
      <div id="vf1" class="content-pane" style="display:none;">
        {regional_fig1}
      </div>
      <div id="vf8" class="content-pane" style="display:none;">
        {graph_html28}
      </div>
      <div id="vf0" class="content-pane" style="display:none;">
        {joined_reig_graphs_Forward_PE}
      </div>
      <div id="vf9" class="content-pane" style="display:none;">
        {joined_reig_graphs_CAPE}
      </div>
      <div id="vf2" class="content-pane" style="display:none;">
        {joined_reig_graphs_Price_to_Book}
      </div>
      <div id="vf3" class="content-pane" style="display:none;">
        {joined_reig_graphs_Price_to_Sales}
      </div>
      <div id="vf4" class="content-pane" style="display:none;">
        {joined_reig_graphs_EV_Trailing_EBITDA}
      </div>
      <div id="vf5" class="content-pane" style="display:none;">
        {joined_reig_graphs_Valuation_Composite}
      </div>
      <div id="vf6" class="content-pane" style="display:none;">
        {cross_sectional_time_series_graph}
      </div>
      <div id="vf7" class="content-pane" style="display:none;">
        {graph_html2}
      </div>

      <!-- Factor Equity Valuations (Broad Category 4) -->
      <div id="vaf1" class="content-pane" style="display:none;">
        {factor_fig1}
      </div>
      <div id="vaf8" class="content-pane" style="display:none;">
        {joined_factor_graphs_Forward_PE}
      </div>
      <div id="vaf9" class="content-pane" style="display:none;">
        {joined_factor_graphs_CAPE}
      </div>
      <div id="vaf2" class="content-pane" style="display:none;">
        {joined_factor_graphs_Price_to_Book }
      </div>
      <div id="vaf3" class="content-pane" style="display:none;">
        {joined_factor_graphs_Price_to_Sales}
      </div>
      <div id="vaf4" class="content-pane" style="display:none;">
        {joined_factor_graphs_EV_Trailing_EBITDA}
      </div>
      <div id="vaf5" class="content-pane" style="display:none;">
        {joined_factor_graphs_Valuation_Composite}
      </div>
      <div id="vaf6" class="content-pane" style="display:none;">
        {graph_html38}
      </div>
      
      <!-- Earnings & Fundamentals (Broad Category 5) -->
      <div id="vvf1" class="content-pane" style="display:none;">
        {blend4q_earnings_rev}
      </div>
      <div id="vvf9" class="content-pane" style="display:none;">
        {graph_html39}
      </div>
      <div id="vvf2" class="content-pane" style="display:none;">
        {graph_html7}
      </div>
      <div id="vvf3" class="content-pane" style="display:none;">
        {graph_html8}
      </div>
      <div id="vvf4" class="content-pane" style="display:none;">
        {graph_html9}
      </div>
      <div id="vvf5" class="content-pane" style="display:none;">
        {graph_html10}
      </div>
      <div id="vvf6" class="content-pane" style="display:none;">
        {graph_html26}
      </div>
      <div id="vvf7" class="content-pane" style="display:none;">
        {graph_html27}
      </div>
      <div id="vvf8" class="content-pane" style="display:none;">
        {graph_html28}
      </div>
      
      <!-- Macroeconomic (Broad Category 6) -->
      <div id="macro1" class="content-pane" style="display:none;">
        <div>{gdp_consensus_html}</div>
        <div>{lei_z_table}</div>
      </div>
      <div id="macro2" class="content-pane" style="display:none;">
        {manufac}
      </div>
      <div id="macro3" class="content-pane" style="display:none;">
        {eco_surpris_df_html}
      </div>
      <div id="macro4" class="content-pane" style="display:none;">
        {graph_html14}
      </div>
      <div id="macro5" class="content-pane" style="display:none;">
        {graph_html15}
      </div>
      <div id="macro6" class="content-pane" style="display:none;">
        {graph_corr_stockbond}
      </div>
      <div id="macro7" class="content-pane" style="display:none;">
        {graph_html30}
      </div>
      <div id="macro8" class="content-pane" style="display:none;">
        {graph_html31}
      </div>
      <div id="macro9" class="content-pane" style="display:none;">
        {graph_html32}
      </div>
      <div id="macro10" class="content-pane" style="display:none;">
        {graph_html33}
      </div>
      <div id="macro11" class="content-pane" style="display:none;">
        {graph_html34}
      </div>
      
      <!-- Technicals / Sentiment / Risk Aversion (Broad Category 7) -->
      <div id="tech1" class="content-pane" style="display:none;">
        {technicals_graphs_html}
      </div>
      <div id="tech2" class="content-pane" style="display:none;">
        {graph_html17}
      </div>
      <div id="tech3" class="content-pane" style="display:none;">
        {graph_html18}
      </div>
      <div id="tech4" class="content-pane" style="display:none;">
        {graph_html19}
      </div>
      <div id="tech5" class="content-pane" style="display:none;">
        {graph_html20}
      </div>
      
      <!-- Bond Futures and Yield Curve (Broad Category 8) -->
      <div id="misc1" class="content-pane" style="display:none;">
        {rate_futures_html}
      </div>
      <div id="misc2" class="content-pane" style="display:none;">
        <div>{US_chart_html}</div>
        <div>{AU_chart_html}</div>
        <div>{EU_chart_html}</div>
        <div>{Globalhedged_chart_html}</div>
        <div>{Global_chart_html}</div>
      </div>
      <div id="misc3" class="content-pane" style="display:none;">
        <div>{Aus_comp_chart_html}</div>
        <div>{Aus_cred_chart_html}</div>
        <div>{Aus_FRN_chart_html}</div>
      </div>
      <div id="misc4" class="content-pane" style="display:none;">
        <div>{US_corp_chart_html}</div>
        <div>{US_cred_chart_html}</div>
      </div>
      <div id="misc5" class="content-pane" style="display:none;">
        {graph_html25}
      </div>
    </div>
  </div>
  
  <!-- jQuery and Bootstrap JS -->
  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
  <script>
  
    // Function to display the selected content pane and hide others
    function showContent(contentId) {{
      var panes = document.getElementsByClassName('content-pane');
      for (var i = 0; i < panes.length; i++) {{
        panes[i].style.display = 'none';
      }}
      document.getElementById(contentId).style.display = 'block';
    }}
    // Show the landing page on page load
    document.addEventListener("DOMContentLoaded", function() {{
      showContent('landing');
    }});

    
  </script>
</body>
</html>
"""
# Write the HTML to a file
with open("github/index.html", "w") as file:
    file.write(html_template)

NEW:

In [ ]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Innova Asset Mangagement Internal Dashboard</title>
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700&display=swap" rel="stylesheet">
  <!-- Bootstrap CSS -->
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    /* Base styles */
    body {{
      margin: 0;
      font-family: 'Montserrat', sans-serif;
    }}
    /* Header Section */
    .site-header {{
      width: 100%;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}
    .palette {{
      display: flex;
    }}
    .palette .color-block {{
      flex: 1;
      height: 8px;
    }}
    .header-title-container {{
      background-color: #30415f;
      padding: 10px;
      text-align: center;
      color: #fff;
      position: relative;
    }}
    /* Home button styling */
    .home-button {{
      position: absolute;
      right: 20px;
      top: 10px;
    }}
    /* Main container for sidebar + content */
    .main-container {{
      display: flex;
      height: calc(100vh - 60px); /* Adjust based on header height */
    }}
    .sidebar {{
      width: 250px;
      background-color: #f5f5f5;
      padding: 10px;
      border-right: 1px solid #ddd;
      overflow-y: auto;
    }}
    .content {{
      flex-grow: 1;
      padding: 20px;
      overflow-y: auto;
    }}
    /* First-level tabs styling for sidebar (unchanged) */
    #sidebar-menu > a.list-group-item {{
      background-color: #30415f;
      color: #fff;
      font-weight: bold;
      border: none;
    }}
    #sidebar-menu > a.list-group-item:hover {{
      background-color: #30415f; 
    }}
    /* Sub-tabs remain default */
    .list-group-item {{
      cursor: pointer;
    }}
    /* Image styling */

    .content-pane div.js-plotly-plot {{
        min-height: 400px;
      }}
      
    .content-pane img {{
      display: block;
      margin: 20px auto; /* Adds spacing between images */
      max-width: 100%; /* Ensures images don’t overflow */
    }}

    /* Graph styling: center divs, canvases, iframes, and svgs inside .content-pane */
    .content-pane div,
    .content-pane canvas,
    .content-pane iframe,
    .content-pane svg {{
      display: block;
      margin: 20px auto; /* Centers the element */
      max-width: 100%;   /* Prevents overflow */
    }}

    /* Horizontal navigation tabs styling */
    .main-nav {{
      margin-bottom: 0;
    }}
    .nav-tabs .nav-link {{
      color: #30415f;  /* Changed text color */
    }}
  </style>
</head>
<body>
  <!-- Header Section with Palette and Title -->
  <header class="site-header">
    <div class="palette">
      <div class="color-block" style="background-color: #30415f;"></div>
      <div class="color-block" style="background-color: #30415f;"></div>
    </div>
    <div class="header-title-container">
      <h1>Innova Asset Mangagement Internal Dashboard</h1>
      <a href="#" class="btn btn-light home-button" onclick="showContent('landing')">Home</a>
    </div>
  </header>

  <!-- Main Tabs Horizontal Navigation -->
  <nav class="main-nav">
    <ul class="nav nav-tabs">
      <li class="nav-item">
        <a class="nav-link" data-main-tab="innova" href="#" onclick="showMainTab('innova')">Innova SMA Analytics</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="dmw" href="#" onclick="showMainTab('dmw')">Daily Indicators</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vf" href="#" onclick="showMainTab('vf')">Region Equity Valuations</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vaf" href="#" onclick="showMainTab('vaf')">Factor Equity Valuations</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vvf" href="#" onclick="showMainTab('vvf')">Earnings & Fundamentals</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="macro" href="#" onclick="showMainTab('macro')">Macroeconomic</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="tech" href="#" onclick="showMainTab('tech')">Technicals / Sentiment / Risk Aversion</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="misc" href="#" onclick="showMainTab('misc')">Bond Futures and Yield Curve</a>
      </li>
    </ul>
  </nav>
  
  <!-- Main Container: Sidebar (Subtabs) + Content Area -->
  <div class="main-container">
    <!-- Sidebar for Subtabs -->
    <div class="sidebar" id="subtab-sidebar">
      <!-- Innova SMA Analytics Subtabs -->
      <div class="subtabs" id="subtabs-innova" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmww1')">Asset Class Views</a>
          <a class="list-group-item" onclick="showContent('dmww2')">Systematic Outputs</a>
          <a class="list-group-item" onclick="showContent('dmww3')">Target Spreadsheets</a>
          <a class="list-group-item" onclick="showContent('dmww4')">Evergreen Innova Charts 2025</a>
          <a class="list-group-item" onclick="showContent('dmww5')">Positioning</a>
        </div>
      </div>
      <!-- Daily Market Watch Subtabs -->
      <div class="subtabs" id="subtabs-dmw" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmw1')">Table Format</a>
          <a class="list-group-item" onclick="showContent('dmw2')">Graph Format</a>
          <a class="list-group-item" onclick="showContent('dmw3')">Factor Performance</a>
          <a class="list-group-item" onclick="showContent('dmw4')">Sector Performance</a>
          <a class="list-group-item" onclick="showContent('dmw5')">Daily Global Valuations</a>
        </div>
      </div>
      <!-- Region Equity Valuations Subtabs -->
      <div class="subtabs" id="subtabs-vf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vf1')">Daily Valuations on Popular Benchmarks</a>
          <a class="list-group-item" onclick="showContent('vf8')">Dedicated Aussie Valuation</a>
          <a class="list-group-item" onclick="showContent('vf0')">Forward PE</a>
          <a class="list-group-item" onclick="showContent('vf9')">CAPE Ratio (Bloomberg Calc)</a>
          <a class="list-group-item" onclick="showContent('vf2')">Price to Book</a>
          <a class="list-group-item" onclick="showContent('vf3')">Price to 5Y Sales</a>
          <a class="list-group-item" onclick="showContent('vf4')">EV to EBITDA</a>
          <a class="list-group-item" onclick="showContent('vf5')">Composite Valuation</a>
          <a class="list-group-item" onclick="showContent('vf6')">Cross-Sectional Valuation</a>
          <a class="list-group-item" onclick="showContent('vf7')">REITs Valuation</a>
        </div>
      </div>
      <!-- Factor Equity Valuations Subtabs -->
      <div class="subtabs" id="subtabs-vaf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vaf1')">Global Factor Matrix</a>
          <a class="list-group-item" onclick="showContent('vaf8')">Forward PE</a>
          <a class="list-group-item" onclick="showContent('vaf9')">CAPE Ratio (Bloomberg Calc)</a>
          <a class="list-group-item" onclick="showContent('vaf2')">Price to Book</a>
          <a class="list-group-item" onclick="showContent('vaf3')">Price to 5Y Sales</a>
          <a class="list-group-item" onclick="showContent('vaf4')">EV to EBITDA</a>
          <a class="list-group-item" onclick="showContent('vaf5')">Composite Valuation</a>
          <a class="list-group-item" onclick="showContent('vaf6')">Cross-Sectional Valuation</a>
        </div>
      </div>
      <!-- Earnings & Fundamentals Subtabs -->
      <div class="subtabs" id="subtabs-vvf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vvf1')">Regional Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf9')">Magnificent 7 Earnings & Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf2')">Factor Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf3')">Sources of Return & Box Valuations</a>
          <a class="list-group-item" onclick="showContent('vvf4')">Key Company Earnings</a>
          <a class="list-group-item" onclick="showContent('vvf5')">S&P Profit Margins</a>
          <a class="list-group-item" onclick="showContent('vvf6')">Global Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf7')">Australia Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf8')">Analyst Estimate... list?</a>
        </div>
      </div>
      <!-- Macroeconomic Subtabs -->
      <div class="subtabs" id="subtabs-macro" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('macro1')">Macroeconomic Regime</a>
          <a class="list-group-item" onclick="showContent('macro2')">US Growth</a>
          <a class="list-group-item" onclick="showContent('macro3')">US Inflation</a>
          <a class="list-group-item" onclick="showContent('macro4')">US Labour Market</a>
          <a class="list-group-item" onclick="showContent('macro5')">US Recession Probability</a>
          <a class="list-group-item" onclick="showContent('macro6')">US Stock Bond Correlation</a>
          <a class="list-group-item" onclick="showContent('macro7')">Domestic Growth</a>
          <a class="list-group-item" onclick="showContent('macro8')">Domestic Inflation</a>
          <a class="list-group-item" onclick="showContent('macro9')">Domestic Labour Market</a>
          <a class="list-group-item" onclick="showContent('macro10')">Global Economic Matrix (Growth/Inf)</a>
          <a class="list-group-item" onclick="showContent('macro11')">Global Liquidity</a>
        </div>
      </div>
      <!-- Technicals / Sentiment / Risk Aversion Subtabs -->
      <div class="subtabs" id="subtabs-tech" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('tech1')">Moving Day Average Graphs</a>
          <a class="list-group-item" onclick="showContent('tech2')">Table of Index Technicals</a>
          <a class="list-group-item" onclick="showContent('tech3')">Volatility of Asset Classes</a>
          <a class="list-group-item" onclick="showContent('tech4')">Net Flows</a>
          <a class="list-group-item" onclick="showContent('tech5')">Fear / Greed Type Indicators</a>
        </div>
      </div>
      <!-- Bond Futures and Yield Curve Subtabs -->
      <div class="subtabs" id="subtabs-misc" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('misc1')">Rate Cut/Hike Pricing</a>
          <a class="list-group-item" onclick="showContent('misc2')">Yield Curves</a>
          <a class="list-group-item" onclick="showContent('misc3')">AU Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc4')">Global Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc5')">TIPS</a>
        </div>
      </div>
    </div>
    
    <!-- Content Area for Charts -->
    <div class="content" id="content-area">
      <!-- Landing Page -->
      <div id="landing" class="content-pane">
        <h2>Welcome Innova Team</h2>
        <p>Please use the tabs above to navigate the different sections. Below are some relevant weekly updates as of {today_date} in macro/markets:</p>
        <ul>
          <li><a href="https://www.cnbc.com/finance/" target="_blank">CNBC Finance Top Headlines for {today_date}</a></li>
          <li> <a href="https://tradingeconomics.com/calendar" target="_blank">Economic Calender/Releases as of {today_date}</a></li>
          <br>
          <br>
          <h2>Top News of the Week:</h2>
          <li>Donald Trump and Vladimir Putin will speak Tuesday as the US pushes for a Ukraine ceasefire. Ahead of the talks, which Trump said will focus on territory, Putin issued a decree allowing some American and other Western funds to sell Russian securities. </li>
          <li>BYD unveiled a battery and charging system that was capable of providing 470 kilometers of range in five minutes in tests on its new Han L sedan. US auto giants have feared a world of Chinese dominance for decades.</li>
          <li>Intel shares rose ~30% after Reuters reported incoming CEO Lip-Bu Tan has considered significant changes to its chip manufacturing methods and AI strategies..</li>
          <li><a href="https://www.atlantafed.org/cqer/research/gdpnow" target="_blank">Latest GDP Nowcast is {next_24_chars}</a></li>
          <li>{crsp_dimensional}</li>
        </ul>
      </div>
      
      <!-- Innova SMA Analytics Content Panes -->
      <div id="dmww1" class="content-pane" style="display:none;">
        <div>{Our_Asset_Class_Views}</div>
      </div>
      <div id="dmww2" class="content-pane" style="display:none;">
        <div>{sys_outputs}</div>
        <div>{recession}</div>
      </div>
      <div id="dmww3" class="content-pane" style="display:none;">
        <div>{Funda}</div>
        <div>{Fla}</div>
        <div>{Trad}</div>
        <div>{Cfs_fc}</div>
      </div>
      <div id="dmww4" class="content-pane" style="display:none;">
        <div>{basecase}</div>
        <div>{eco_surprise}</div>
        <div>{concentration}</div>
        <div>{cape_chart}</div>
        <div>{returns_2022}</div>
      </div>
      <div id="dmww5" class="content-pane" style="display:none;">
        {region_positioning}
      </div>

      <!-- Daily Market Watch Content Panes -->
      <div id="dmw1" class="content-pane" style="display:none;">
       <h2 style="text-align: center;">'Equities only'</h2>
       <div style="display: flex; justify-content: center; margin-top: 20px;">
        {equity_styled}
        </div>
        <h2>{'Fixed Income only'}</h2>
        <div>{debt_styled}</div>
        <h2>{'Other'}</h2>
        <div>{other_styled}</div>
      </div>
      <div id="dmw2" class="content-pane" style="display:none;">
        {graph_for_regional_equity_ytd}
      </div>
      <div id="dmw3" class="content-pane" style="display:none;">
        <div>{graph_for_factor_equity_ytd}</div>
        <div class="plot-container">{graph_for_factor_equity}</div>
      </div>
      <div id="dmw4" class="content-pane" style="display:none;">
        <h2>{'US Sector'}</h2>
        <div>{us_sector1}</div>
        <h2>{'AU Sector'}</h2>
        <div>{au_sector1}</div>
        <h2>{'Japan Sector'}</h2>
        <div>{jap_sector1}</div>
        <h2>{'EU Sector'}</h2>
        <div>{eu_sector1}</div>
        <h2>{'UK Sector'}</h2>
        <div>{uk_sector1}</div>
      </div>
      <div id="dmw5" class="content-pane" style="display:none;">
        <div>{together_daily_ht}</div>
        <div>{sp500_daily}</div>
        <div>{eur_daily}</div>
        <div>{asx_daily}</div>
        <div>{nky_daily}</div>
      </div>
      
      <!-- Region Equity Valuations Content Panes -->
      <div id="vf1" class="content-pane" style="display:none;">
        {regional_fig1}
      </div>
      <div id="vf8" class="content-pane" style="display:none;">
        <div>{aussie_blend4q_earnings_rev}</div>
        <div>{aus_joined_reig_graphs_Valuation_Composite}</div>
        <div>{aus_joined_reig_graphs_Forward_PE}</div>
        <div>{aus_joined_reig_graphs_CAPE}</div>
        <div>{aus_joined_reig_graphs_Price_to_Book}</div>
        <div>{aus_joined_reig_graphs_Price_to_Sales}</div>
          
      </div>
      <div id="vf0" class="content-pane" style="display:none;">
        {joined_reig_graphs_Forward_PE}
      </div>
      <div id="vf9" class="content-pane" style="display:none;">
        {joined_reig_graphs_CAPE}
      </div>
      <div id="vf2" class="content-pane" style="display:none;">
        {joined_reig_graphs_Price_to_Book}
      </div>
      <div id="vf3" class="content-pane" style="display:none;">
        {joined_reig_graphs_Price_to_Sales}
      </div>
      <div id="vf4" class="content-pane" style="display:none;">
        {joined_reig_graphs_EV_Trailing_EBITDA}
      </div>
      <div id="vf5" class="content-pane" style="display:none;">
        {joined_reig_graphs_Valuation_Composite}
      </div>
      <div id="vf6" class="content-pane" style="display:none;">
        {cross_sectional_time_series_graph}
      </div>
      <div id="vf7" class="content-pane" style="display:none;">
        {graph_html2}
      </div>

      <!-- Factor Equity Valuations Content Panes -->
      <div id="vaf1" class="content-pane" style="display:none;">
        {factor_fig1}
      </div>
      <div id="vaf8" class="content-pane" style="display:none;">
        {joined_factor_graphs_Forward_PE}
      </div>
      <div id="vaf9" class="content-pane" style="display:none;">
        {joined_factor_graphs_CAPE}
      </div>
      <div id="vaf2" class="content-pane" style="display:none;">
        {joined_factor_graphs_Price_to_Book }
      </div>
      <div id="vaf3" class="content-pane" style="display:none;">
        {joined_factor_graphs_Price_to_Sales}
      </div>
      <div id="vaf4" class="content-pane" style="display:none;">
        {joined_factor_graphs_EV_Trailing_EBITDA}
      </div>
      <div id="vaf5" class="content-pane" style="display:none;">
        {joined_factor_graphs_Valuation_Composite}
      </div>
      <div id="vaf6" class="content-pane" style="display:none;">
        {graph_html38}
      </div>
      
      <!-- Earnings & Fundamentals Content Panes -->
      <div id="vvf1" class="content-pane" style="display:none;">
        {blend4q_earnings_rev}
      </div>
      <div id="vvf9" class="content-pane" style="display:none;">
        {graph_html39}
      </div>
      <div id="vvf2" class="content-pane" style="display:none;">
        {graph_html7}
      </div>
      <div id="vvf3" class="content-pane" style="display:none;">
        {graph_html8}
      </div>
      <div id="vvf4" class="content-pane" style="display:none;">
        {graph_html9}
      </div>
      <div id="vvf5" class="content-pane" style="display:none;">
        {graph_html10}
      </div>
      <div id="vvf6" class="content-pane" style="display:none;">
        {graph_html26}
      </div>
      <div id="vvf7" class="content-pane" style="display:none;">
        {graph_html27}
      </div>
      <div id="vvf8" class="content-pane" style="display:none;">
        {graph_html28}
      </div>
      
      <!-- Macroeconomic Content Panes -->
      <div id="macro1" class="content-pane" style="display:none;">
        <div>{gdp_consensus_html}</div>
        <div>{lei_z_table}</div>
      </div>
      <div id="macro2" class="content-pane" style="display:none;">
        {manufac}
      </div>
      <div id="macro3" class="content-pane" style="display:none;">
        {eco_surpris_df_html}
      </div>
      <div id="macro4" class="content-pane" style="display:none;">
        {deficit_matrix}
      </div>
      <div id="macro5" class="content-pane" style="display:none;">
        {graph_html15}
      </div>
      <div id="macro6" class="content-pane" style="display:none;">
        {graph_corr_stockbond}
      </div>
      <div id="macro7" class="content-pane" style="display:none;">
        {graph_html30}
      </div>
      <div id="macro8" class="content-pane" style="display:none;">
        {graph_html31}
      </div>
      <div id="macro9" class="content-pane" style="display:none;">
        {graph_html32}
      </div>
      <div id="macro10" class="content-pane" style="display:none;">
        {graph_html33}
      </div>
      <div id="macro11" class="content-pane" style="display:none;">
        {graph_html34}
      </div>
      
      <!-- Technicals / Sentiment / Risk Aversion Content Panes -->
      <div id="tech1" class="content-pane" style="display:none;">
        {technicals_graphs_html}
      </div>
      <div id="tech2" class="content-pane" style="display:none;">
        {graph_html17}
      </div>
      <div id="tech3" class="content-pane" style="display:none;">
        {graph_html18}
      </div>
      <div id="tech4" class="content-pane" style="display:none;">
        {graph_html19}
      </div>
      <div id="tech5" class="content-pane" style="display:none;">
        {graph_html20}
      </div>
      
      <!-- Bond Futures and Yield Curve Content Panes -->
      <div id="misc1" class="content-pane" style="display:none;">
        <div>{rate_futures_html}</div>
        <div>{ten_10y_decomp_html}</div>  
      </div>
      <div id="misc2" class="content-pane" style="display:none;">
        <div>{US_chart_html}</div>
        <div>{AU_chart_html}</div>
        <div>{EU_chart_html}</div>
        <div>{Globalhedged_chart_html}</div>
        <div>{Global_chart_html}</div>
      </div>
      <div id="misc3" class="content-pane" style="display:none;">
        <div>{Aus_comp_chart_html}</div>
        <div>{Aus_cred_chart_html}</div>
        <div>{Aus_FRN_chart_html}</div>
      </div>
      <div id="misc4" class="content-pane" style="display:none;">
        <div>{US_corp_chart_html}</div>
        <div>{US_cred_chart_html}</div>
      </div>
      <div id="misc5" class="content-pane" style="display:none;">
        {tips_html}
      </div>
    </div>
  </div>
  
  <!-- jQuery and Bootstrap JS -->
  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> <!-- Add Plotly CDN here -->
  <script>
  // Function to display the selected content pane and hide others
  function showContent(contentId) {{
    var panes = document.getElementsByClassName('content-pane');
    for (var i = 0; i < panes.length; i++) {{
      panes[i].style.display = 'none';
    }}

    var el = document.getElementById(contentId);
    if (el) {{
      el.style.display = 'block';

      // Resize Plotly charts after showing the content
      setTimeout(function () {{
        var plotlyGraphs = el.querySelectorAll('.js-plotly-plot');
        for (var j = 0; j < plotlyGraphs.length; j++) {{
          if (typeof Plotly !== 'undefined') {{
            Plotly.Plots.resize(plotlyGraphs[j]);
          }}
        }}
      }}, 600);
    }}
  }}

  // Function to display the corresponding subtab menu based on the selected main tab
  function showMainTab(tabId) {{
    var subtabs = document.getElementsByClassName('subtabs');
    for (var i = 0; i < subtabs.length; i++) {{
      subtabs[i].style.display = 'none';
    }}

    var currentSubtab = document.getElementById('subtabs-' + tabId);
    if (currentSubtab) {{
      currentSubtab.style.display = 'block';
    }}

    showContent('landing');

    var mainTabLinks = document.querySelectorAll('.nav-tabs .nav-link');
    for (var i = 0; i < mainTabLinks.length; i++) {{
      mainTabLinks[i].classList.remove('active');
    }}

    var activeLink = document.querySelector('.nav-tabs .nav-link[data-main-tab=\"' + tabId + '\"]');
    if (activeLink) {{
      activeLink.classList.add('active');
    }}
  }}

  document.addEventListener("DOMContentLoaded", function () {{
    showMainTab('innova');
    showContent('landing');
  }});
</script>
  
</body>
</html>
"""
# Write the HTML to a file
with open("github/index.html", "w", encoding='utf-8') as file:
    file.write(html_template)

W/ Drop down..draft:

In [89]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Innova Asset Mangagement Internal Dashboard</title>
  <!-- Google Fonts -->
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700&display=swap" rel="stylesheet">
  <!-- Bootstrap CSS -->
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    /* Base styles */
    body {{
      margin: 0;
      font-family: 'Montserrat', sans-serif;
    }}
    /* Header Section */
    .site-header {{
      width: 100%;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}
    .palette {{
      display: flex;
    }}
    .palette .color-block {{
      flex: 1;
      height: 8px;
    }}
    .header-title-container {{
      background-color: #30415f;
      padding: 10px;
      text-align: center;
      color: #fff;
      position: relative;
    }}
    /* Home button styling */
    .home-button {{
      position: absolute;
      right: 20px;
      top: 10px;
    }}
    /* Main container for sidebar + content */
    .main-container {{
      display: flex;
      height: calc(100vh - 60px); /* Adjust based on header height */
    }}
    .sidebar {{
      width: 250px;
      background-color: #f5f5f5;
      padding: 10px;
      border-right: 1px solid #ddd;
      overflow-y: auto;
    }}
    .content {{
      flex-grow: 1;
      padding: 20px;
      overflow-y: auto;
    }}
    /* First-level tabs styling for sidebar (unchanged) */
    #sidebar-menu > a.list-group-item {{
      background-color: #30415f;
      color: #fff;
      font-weight: bold;
      border: none;
    }}
    #sidebar-menu > a.list-group-item:hover {{
      background-color: #30415f; 
    }}
    /* Sub-tabs remain default */
    .list-group-item {{
      cursor: pointer;
    }}
    /* Image styling */

    .content-pane div.js-plotly-plot {{
        min-height: 400px;
      }}
      
    .content-pane img {{
      display: block;
      margin: 20px auto; /* Adds spacing between images */
      max-width: 100%; /* Ensures images don’t overflow */
    }}

    /* Graph styling: center divs, canvases, iframes, and svgs inside .content-pane */
    .content-pane div,
    .content-pane canvas,
    .content-pane iframe,
    .content-pane svg {{
      display: block;
      margin: 20px auto; /* Centers the element */
      max-width: 100%;   /* Prevents overflow */
    }}

    /* Horizontal navigation tabs styling */
    .main-nav {{
      margin-bottom: 0;
    }}
    .nav-tabs .nav-link {{
      color: #30415f;  /* Changed text color */
    }}


    /* Dropdown styling */
    .graph-selector {{
      display: block;
      width: 100%;
      max-width: 500px;
      margin: 20px auto;
      padding: 8px 12px;
      font-size: 16px;
      border: 1px solid #ccc;
      border-radius: 4px;
      background-color: white;
      font-family: 'Montserrat', sans-serif;
    }}
    
    /* Graph container */
    .graph-container {{
      margin-top: 30px;
    }}

  
    .sector-table {{
      margin: 0 auto;
      text-align: center;
    }}
    .sector-table th, .sector-table td {{
      text-align: center;
      padding: 8px;
    }}
    </style>

  </style>
</head>
<body>
  <!-- Header Section with Palette and Title -->
  <header class="site-header">
    <div class="palette">
      <div class="color-block" style="background-color: #30415f;"></div>
      <div class="color-block" style="background-color: #30415f;"></div>
    </div>
    <div class="header-title-container">
      <h1>Innova Asset Mangagement Internal Dashboard</h1>
      <a href="#" class="btn btn-light home-button" onclick="showContent('landing')">Home</a>
    </div>
  </header>

  <!-- Main Tabs Horizontal Navigation -->
  <nav class="main-nav">
    <ul class="nav nav-tabs">
      <li class="nav-item">
        <a class="nav-link" data-main-tab="innova" href="#" onclick="showMainTab('innova')">Innova SMA Analytics</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="dmw" href="#" onclick="showMainTab('dmw')">Daily Indicators</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vf" href="#" onclick="showMainTab('vf')">Region Equity Valuations</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vaf" href="#" onclick="showMainTab('vaf')">Factor Equity Valuations</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="vvf" href="#" onclick="showMainTab('vvf')">Earnings & Fundamentals</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="macro" href="#" onclick="showMainTab('macro')">Macroeconomic</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="tech" href="#" onclick="showMainTab('tech')">Technicals / Sentiment / Risk Aversion</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="misc" href="#" onclick="showMainTab('misc')">Bond Futures and Yield Curve</a>
      </li>
    </ul>
  </nav>
  
  <!-- Main Container: Sidebar (Subtabs) + Content Area -->
  <div class="main-container">
    <!-- Sidebar for Subtabs -->
    <div class="sidebar" id="subtab-sidebar">
      <!-- Innova SMA Analytics Subtabs -->
      <div class="subtabs" id="subtabs-innova" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmww1')">Traffic Lights</a>
          <a class="list-group-item" onclick="showContent('dmww3')">Target Spreadsheets</a>
          <a class="list-group-item" onclick="showContent('dmww4')">Innova Charts 2025</a>
        </div>
      </div>
      <!-- Daily Market Watch Subtabs -->
      <div class="subtabs" id="subtabs-dmw" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmw1')">Table Format</a>
          <a class="list-group-item" onclick="showContent('dmw2')">Graph Format</a>
          <a class="list-group-item" onclick="showContent('dmw3')">Factor Performance</a>
          <a class="list-group-item" onclick="showContent('dmw4')">Sector Performance</a>
          <a class="list-group-item" onclick="showContent('dmw5')">Daily Global Valuations</a>
        </div>
      </div>
      <!-- Region Equity Valuations Subtabs -->
      <div class="subtabs" id="subtabs-vf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vf1')">Daily Valuations on Popular Benchmarks</a>
          <a class="list-group-item" onclick="showContent('vf8')">Dedicated Aussie Valuation</a>
          <a class="list-group-item" onclick="showContent('vf0')">Reigonal Time-Series Val</a>
          <a class="list-group-item" onclick="showContent('vf6')">Cross-Sectional Valuation</a>
          <a class="list-group-item" onclick="showContent('vf7')">REITs Valuation</a>
        </div>
      </div>
      <!-- Factor Equity Valuations Subtabs -->
      <div class="subtabs" id="subtabs-vaf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vaf1')">Global Factor Matrix</a>
          <a class="list-group-item" onclick="showContent('vaf8')">Factor Time Series Valuation</a>
          <a class="list-group-item" onclick="showContent('vaf6')">Cross-Sectional Valuation</a>
        </div>
      </div>
      <!-- Earnings & Fundamentals Subtabs -->
      <div class="subtabs" id="subtabs-vvf" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('vvf1')">Regional Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf9')">Magnificent 7 Earnings & Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf2')">Factor Earnings Growth / Revisions</a>
          <a class="list-group-item" onclick="showContent('vvf3')">Sources of Return & Box Valuations</a>
          <a class="list-group-item" onclick="showContent('vvf4')">Key Company Earnings</a>
          <a class="list-group-item" onclick="showContent('vvf5')">S&P Profit Margins</a>
          <a class="list-group-item" onclick="showContent('vvf6')">Global Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf7')">Australia Sector Valuation</a>
          <a class="list-group-item" onclick="showContent('vvf8')">Analyst Estimate... list?</a>
        </div>
      </div>
      <!-- Macroeconomic Subtabs -->
      <div class="subtabs" id="subtabs-macro" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('macro1')">Macroeconomic Regime</a>
          <a class="list-group-item" onclick="showContent('macro2')">US Growth</a>
          <a class="list-group-item" onclick="showContent('macro3')">US Inflation</a>
          <a class="list-group-item" onclick="showContent('macro4')">US Labour Market</a>
          <a class="list-group-item" onclick="showContent('macro5')">US Recession Probability</a>
          <a class="list-group-item" onclick="showContent('macro6')">US Stock Bond Correlation</a>
          <a class="list-group-item" onclick="showContent('macro7')">Domestic Growth</a>
          <a class="list-group-item" onclick="showContent('macro8')">Domestic Inflation</a>
          <a class="list-group-item" onclick="showContent('macro9')">Domestic Labour Market</a>
          <a class="list-group-item" onclick="showContent('macro10')">Global Economic Matrix (Growth/Inf)</a>
          <a class="list-group-item" onclick="showContent('macro11')">Global Liquidity</a>
        </div>
      </div>
      <!-- Technicals / Sentiment / Risk Aversion Subtabs -->
      <div class="subtabs" id="subtabs-tech" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('tech1')">Moving Day Average Graphs</a>
          <a class="list-group-item" onclick="showContent('tech2')">Table of Index Technicals</a>
          <a class="list-group-item" onclick="showContent('tech3')">Volatility of Asset Classes</a>
          <a class="list-group-item" onclick="showContent('tech4')">Net Flows</a>
          <a class="list-group-item" onclick="showContent('tech5')">Fear / Greed Type Indicators</a>
        </div>
      </div>
      <!-- Bond Futures and Yield Curve Subtabs -->
      <div class="subtabs" id="subtabs-misc" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('misc1')">Rate Cut/Hike Pricing</a>
          <a class="list-group-item" onclick="showContent('misc2')">Yield Curves</a>
          <a class="list-group-item" onclick="showContent('misc3')">AU Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc4')">Global Credit Spreads</a>
          <a class="list-group-item" onclick="showContent('misc5')">TIPS</a>
        </div>
      </div>
    </div>
    
    <!-- Content Area for Charts -->
    <div class="content" id="content-area">
      <!-- Landing Page -->
      <div id="landing" class="content-pane">
        <h2>Welcome Innova Team</h2>
        <p>Please use the tabs above to navigate the different sections. Below are some relevant weekly updates as of {today_date} in macro/markets:</p>
        <ul>
          <li><a href="https://www.cnbc.com/finance/" target="_blank">CNBC Finance Top Headlines for {today_date}</a></li>
          <li> <a href="https://tradingeconomics.com/calendar" target="_blank">Economic Calender/Releases as of {today_date}</a></li>
          <br>
          <br>
          <h2>Top News of the Week:</h2>
          <li> JPMorgan noted that the tariffs would hike taxes on Americans by $660 billion a year, the largest tax increase in recent memory by a longshot. It will cause prices to surge, too, adding 2% to the Consumer Price Index, a measure of US inflation that has struggled to come back down to earth in recent years.</li>
          <li></li>
          <li><a href="https://www.atlantafed.org/cqer/research/gdpnow" target="_blank">Latest GDP Nowcast is {next_24_chars}</a></li>
          <div>{tariff_2025}</div>
          <div>{tariff_2025_asia}</div>
        </ul>
      </div>
      
      <!-- Innova SMA Analytics Content Panes -->
      <div id="dmww1" class="content-pane" style="display:none;">
        <div>{aashna_all_asset_class_z_score_valuations_html}</div>
      </div>
      <div id="dmww3" class="content-pane" style="display:none;">
        <div>{Funda}</div>
        <div>{Fla}</div>
        <div>{Trad}</div>
        <div>{Cfs_fc}</div>
      </div>
      <div id="dmww4" class="content-pane" style="display:none;">
        <div>{basecase}</div>
        <div>{eco_surprise}</div>
        <div>{concentration}</div>
        <div>{cape_chart}</div>
        <div>{returns_2022}</div>
        <div>{crsp_dimensional}</div>
        <div>{region_positioning}</div>
      </div>

      <!-- Daily Market Watch Content Panes -->
      <div id="dmw1" class="content-pane" style="display:none;">
       <h2 style="text-align: center;">'Equities only'</h2>
       <div style="display: flex; justify-content: center; margin-top: 20px;">
        {equity_styled}
        </div>
        <h2>{'Fixed Income only'}</h2>
        <div>{debt_styled}</div>
        <h2>{'Other'}</h2>
        <div>{other_styled}</div>
      </div>
      <div id="dmw2" class="content-pane" style="display:none;">
        {graph_for_regional_equity_ytd}
      </div>
      <div id="dmw3" class="content-pane" style="display:none;">
        <div>{graph_for_factor_equity_ytd}</div>
        <div class="plot-container">{graph_for_factor_equity}</div>
      </div>
      <div id="dmw4" class="content-pane" style="display: none; overflow-x: auto;">

        <h2>US Sector</h2>
        <div>{us_sector1}</div>
        <h2>AU Sector</h2>
        <div>{au_sector1}</div>
        <h2>Japan Sector</h2>
        <div>{jap_sector1}</div>
        <h2>EU Sector</h2>
        <div>{eu_sector1}</div>
        <h2>UK Sector</h2>
        <div>{uk_sector1}</div>
      </div>

      <div id="dmw5" class="content-pane" style="display:none;">
        <div>{together_daily_ht}</div>
        <div>{sp500_daily}</div>
        <div>{eur_daily}</div>
        <div>{asx_daily}</div>
        <div>{nky_daily}</div>
      </div>
      
      <!-- Region Equity Valuations Content Panes -->
      <div id="vf1" class="content-pane" style="display:none;">
        {regional_fig1}
      </div>
      <div id="vf8" class="content-pane" style="display:none;">
        <div>{aussie_blend4q_earnings_rev}</div>

        <select id="ausValueSelector" class="graph-selector" onchange="showAusValuation(this.value)">
          <option value="">-- Select a valuation metric --</option>
          <option value="aus_forward_pe">Forward PE</option>
          <option value="aus_cape">CAPE Ratio</option>
          <option value="aus_price_to_book">Price to Book</option>
          <option value="aus_price_to_sales">Price to Sales</option>
          <option value="aus_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
          <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
          </div>
          <div id="aus_forward_pe" style="display: none;">{aus_joined_reig_graphs_Forward_PE}</div>
          <div id="aus_cape" style="display: none;">{aus_joined_reig_graphs_CAPE}</div>
          <div id="aus_price_to_book" style="display: none;">{aus_joined_reig_graphs_Price_to_Book}</div>
          <div id="aus_price_to_sales" style="display: none;">{aus_joined_reig_graphs_Price_to_Sales}</div>
          <div id="aus_valuation_composite" style="display: none;">{aus_joined_reig_graphs_Valuation_Composite}</div>
        </div>
      </div>

      <div id="vf0" class="content-pane" style="display:none;">

        <select id="reigValueSelector" class="graph-selector" onchange="reigAusValuation(this.value)">
          <option value="">-- Select a valuation metric --</option>
          <option value="_forward_pe">Forward PE</option>
          <option value="_cape">CAPE Ratio</option>
          <option value="_price_to_book">Price to Book</option>
          <option value="_price_to_sales">Price to Sales</option>
          <option value="_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
          <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
          </div>
          <div id="_forward_pe" style="display: none;">{joined_reig_graphs_Forward_PE}</div>
          <div id="_cape" style="display: none;">{joined_reig_graphs_CAPE}</div>
          <div id="_price_to_book" style="display: none;">{joined_reig_graphs_Price_to_Book}</div>
          <div id="_price_to_sales" style="display: none;">{joined_reig_graphs_Price_to_Sales}</div>
          <div id="_valuation_composite" style="display: none;">{joined_reig_graphs_Valuation_Composite}</div>
        </div>
      </div>

      <div id="vf6" class="content-pane" style="display:none;">
        {cross_sectional_time_series_graph}
      </div>
      <div id="vf7" class="content-pane" style="display:none;">
        {graph_html2}
      </div>

      <!-- Factor Equity Valuations Content Panes -->
      <div id="vaf1" class="content-pane" style="display:none;">
        {factor_fig1}
      </div>
      <div id="vaf8" class="content-pane" style="display:none;">

        <select id="ValueSelector" class="graph-selector" onchange="showValuation(this.value)">
          <option value="">-- Select a valuation metric --</option>
          <option value="factor_forward_pe">Forward PE</option>
          <option value="factor_cape">CAPE Ratio</option>
          <option value="factor_price_to_book">Price to Book</option>
          <option value="factor_price_to_sales">Price to Sales</option>
          <option value="factor_valuation_composite">Valuation Composite</option>
        </select>

        <div id="graph-container" class="graph-container">
          <div id="valuation-default" style="text-align: center; padding: 40px;">
            <p>Please select a valuation metric from the dropdown above</p>
          </div>
          <div id="factor_forward_pe" style="display: none;">{joined_factor_graphs_Forward_PE}</div>
          <div id="factor_cape" style="display: none;">{joined_factor_graphs_CAPE}</div>
          <div id="factor_price_to_book" style="display: none;">{joined_factor_graphs_Price_to_Book}</div>
          <div id="factor_price_to_sales" style="display: none;">{joined_factor_graphs_Price_to_Sales}</div>
          <div id="factor_valuation_composite" style="display: none;">{joined_factor_graphs_Valuation_Composite}</div>

        </div>
      </div>
      
      <!-- Earnings & Fundamentals Content Panes -->
      <div id="vvf1" class="content-pane" style="display:none;">
        {blend4q_earnings_rev}
      </div>
      <div id="vvf9" class="content-pane" style="display:none;">
        {graph_html39}
      </div>
      <div id="vvf2" class="content-pane" style="display:none;">
        {graph_html7}
      </div>
      <div id="vvf3" class="content-pane" style="display:none;">
        {graph_html8}
      </div>
      <div id="vvf4" class="content-pane" style="display:none;">
        {graph_html9}
      </div>
      <div id="vvf5" class="content-pane" style="display:none;">
        {graph_html10}
      </div>
      <div id="vvf6" class="content-pane" style="display:none;">
        {graph_html26}
      </div>
      <div id="vvf7" class="content-pane" style="display:none;">
        {graph_html27}
      </div>
      <div id="vvf8" class="content-pane" style="display:none;">
        {graph_html28}
      </div>
      
      <!-- Macroeconomic Content Panes -->
      <div id="macro1" class="content-pane" style="display:none;">
        <div>{gdp_consensus_html}</div>
        <div>{lei_z_table}</div>
      </div>
      <div id="macro2" class="content-pane" style="display:none;">
        {manufac}
      </div>
      <div id="macro3" class="content-pane" style="display:none;">
        {eco_surpris_df_html}
      </div>
      <div id="macro4" class="content-pane" style="display:none;">
        {deficit_matrix}
      </div>
      <div id="macro5" class="content-pane" style="display:none;">
        {graph_html15}
      </div>
      <div id="macro6" class="content-pane" style="display:none;">
        {graph_corr_stockbond}
      </div>
      <div id="macro7" class="content-pane" style="display:none;">
        {graph_html30}
      </div>
      <div id="macro8" class="content-pane" style="display:none;">
        {graph_html31}
      </div>
      <div id="macro9" class="content-pane" style="display:none;">
        {graph_html32}
      </div>
      <div id="macro10" class="content-pane" style="display:none;">
        {graph_html33}
      </div>
      <div id="macro11" class="content-pane" style="display:none;">
        {graph_html34}
      </div>
      
      <!-- Technicals / Sentiment / Risk Aversion Content Panes -->
      <div id="tech1" class="content-pane" style="display:none;">
        {technicals_graphs_html}
      </div>
      <div id="tech2" class="content-pane" style="display:none;">
        {graph_html17}
      </div>
      <div id="tech3" class="content-pane" style="display:none;">
        {graph_html18}
      </div>
      <div id="tech4" class="content-pane" style="display:none;">
        {graph_html19}
      </div>
      <div id="tech5" class="content-pane" style="display:none;">
        {graph_html20}
      </div>
      
      <!-- Bond Futures and Yield Curve Content Panes -->
      <div id="misc1" class="content-pane" style="display:none;">
        <div>{rate_futures_html}</div>
        <div>{ten_10y_decomp_html}</div>  
      </div>
      <div id="misc2" class="content-pane" style="display:none;">
        <div>{US_chart_html}</div>
        <div>{AU_chart_html}</div>
        <div>{EU_chart_html}</div>
        <div>{Globalhedged_chart_html}</div>
        <div>{Global_chart_html}</div>
      </div>
      <div id="misc3" class="content-pane" style="display:none;">
        <div>{Aus_comp_chart_html}</div>
        <div>{Aus_cred_chart_html}</div>
        <div>{Aus_FRN_chart_html}</div>
      </div>
      <div id="misc4" class="content-pane" style="display:none;">
        <div>{US_corp_chart_html}</div>
        <div>{US_cred_chart_html}</div>
      </div>
      <div id="misc5" class="content-pane" style="display:none;">
        {tips_html}
      </div>
    </div>
  </div>
  
  <!-- jQuery and Bootstrap JS -->
  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>
  <script src="https://cdn.plot.ly/plotly-latest.min.js"></script> <!-- Add Plotly CDN here -->
  <script>

    function showValuationDropdown(metricId, ids) {{
      ids.forEach(id => {{
        const el = document.getElementById(id);
        if (el) el.style.display = "none";
      }});

      const selected = document.getElementById(metricId);
      const defaultMsg = document.getElementById("valuation-default");
      if (selected) {{
        selected.style.display = "block";
        if (defaultMsg) defaultMsg.style.display = "none";
      }} else {{
        if (defaultMsg) defaultMsg.style.display = "block";
      }}

      if (selected) {{
        setTimeout(() => {{
          const graphs = selected.querySelectorAll('.js-plotly-plot');
          graphs.forEach(graph => {{
            if (typeof Plotly !== 'undefined') {{
              Plotly.Plots.resize(graph);
            }}
          }});
        }}, 600);
      }}
    }}

    function showAusValuation(metricId) {{
      showValuationDropdown(metricId, [
        "aus_forward_pe", "aus_cape", "aus_price_to_book", "aus_price_to_sales", "aus_valuation_composite"
      ]);
    }}

    function reigAusValuation(metricId) {{
      showValuationDropdown(metricId, [
        "_forward_pe", "_cape", "_price_to_book", "_price_to_sales", "_valuation_composite"
      ]);
    }}

    function showValuation(metricId) {{
      showValuationDropdown(metricId, [
        "factor_forward_pe", "factor_cape", "factor_price_to_book", "factor_price_to_sales", "factor_valuation_composite"
      ]);
    }}


  // Function to display the corresponding subtab menu based on the selected main tab
  function showMainTab(tabId) {{
    // Hide all subtabs first
    var subtabs = document.getElementsByClassName('subtabs');
    for (var i = 0; i < subtabs.length; i++) {{
      subtabs[i].style.display = 'none';
    }}

    // Show current tab’s subtab section
    var currentSubtab = document.getElementById('subtabs-' + tabId);
    if (currentSubtab) {{
      currentSubtab.style.display = 'block';
    }}

    // Automatically open the first subtab under the selected main tab
    const firstSubtab = document.querySelector(`#subtabs-${{tabId}} .list-group-item`);
    if (firstSubtab && firstSubtab.getAttribute('onclick')) {{
      const onclickContentId = firstSubtab.getAttribute('onclick').match(/showContent\\('(.+?)'\\)/);
      if (onclickContentId && onclickContentId[1]) {{
        showContent(onclickContentId[1]);
      }}
    }}

    // Tab highlighting logic
    var mainTabLinks = document.querySelectorAll('.nav-tabs .nav-link');
    for (var i = 0; i < mainTabLinks.length; i++) {{
      mainTabLinks[i].classList.remove('active');
    }}

    var activeLink = document.querySelector('.nav-tabs .nav-link[data-main-tab="{{' + tabId + '}}"]');
    if (activeLink) {{
      activeLink.classList.add('active');
    }}
  }}

  document.addEventListener("DOMContentLoaded", function () {{
    showMainTab('innova');
    showContent('landing');
  }});

  
  function showContent(contentId) {{
    var panes = document.getElementsByClassName('content-pane');
    for (var i = 0; i < panes.length; i++) {{
      panes[i].style.display = 'none';
    }}

    var el = document.getElementById(contentId);
    if (el) {{
      el.style.display = 'block';

      // Resize Plotly charts after showing the content
      setTimeout(function () {{
        var plotlyGraphs = el.querySelectorAll('.js-plotly-plot');
        for (var j = 0; j < plotlyGraphs.length; j++) {{
          if (typeof Plotly !== 'undefined') {{
            Plotly.Plots.resize(plotlyGraphs[j]);
          }}
        }}
      }}, 600);
    }}
  }}

</script>
  
</body>
</html>
"""
# Write the HTML to a file
with open("github/index.html", "w", encoding='utf-8') as file:
    file.write(html_template)

In [73]:
html_template = f"""
<!DOCTYPE html>
<html>
<head>
  <meta charset="UTF-8">
  <title>Innova Asset Management Internal Dashboard</title>
  <link href="https://fonts.googleapis.com/css?family=Montserrat:400,700&display=swap" rel="stylesheet">
  <link rel="stylesheet" href="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/css/bootstrap.min.css">
  <style>
    body {{
      margin: 0;
      font-family: 'Montserrat', sans-serif;
    }}
    .site-header {{
      width: 100%;
      box-shadow: 0 2px 4px rgba(0,0,0,0.1);
    }}
    .palette {{
      display: flex;
    }}
    .palette .color-block {{
      flex: 1;
      height: 8px;
    }}
    .header-title-container {{
      background-color: #30415f;
      padding: 10px;
      text-align: center;
      color: #fff;
      position: relative;
    }}
    .home-button {{
      position: absolute;
      right: 20px;
      top: 10px;
    }}
    .main-container {{
      display: flex;
      height: calc(100vh - 60px);
    }}
    .sidebar {{
      width: 250px;
      background-color: #f5f5f5;
      padding: 10px;
      border-right: 1px solid #ddd;
      overflow-y: auto;
    }}
    .content {{
      flex-grow: 1;
      padding: 20px;
      overflow-y: auto;
    }}
    #sidebar-menu > a.list-group-item {{
      background-color: #30415f;
      color: #fff;
      font-weight: bold;
      border: none;
    }}
    #sidebar-menu > a.list-group-item:hover {{
      background-color: #30415f;
    }}
    .list-group-item {{
      cursor: pointer;
    }}
    .content-pane img {{
      display: block;
      margin: 20px auto;
      max-width: 100%;
    }}
    .content-pane div,
    .content-pane canvas,
    .content-pane iframe,
    .content-pane svg {{
      display: block;
      margin: 20px auto;
      max-width: 100%;
    }}
    .main-nav {{
      margin-bottom: 0;
    }}
    .nav-tabs .nav-link {{
      color: #30415f;
    }}
  </style>
</head>
<body>
  <header class="site-header">
    <div class="palette">
      <div class="color-block" style="background-color: #30415f;"></div>
      <div class="color-block" style="background-color: #30415f;"></div>
    </div>
    <div class="header-title-container">
      <h1>Innova Asset Management Internal Dashboard</h1>
      <a href="#" class="btn btn-light home-button" onclick="showContent('landing')">Home</a>
    </div>
  </header>

  <nav class="main-nav">
    <ul class="nav nav-tabs">
      <li class="nav-item">
        <a class="nav-link" data-main-tab="innova" href="#" onclick="showMainTab('innova')">Innova SMA Analytics</a>
      </li>
      <li class="nav-item">
        <a class="nav-link" data-main-tab="dmw" href="#" onclick="showMainTab('dmw')">Daily Indicators</a>
      </li>
    </ul>
  </nav>

  <div class="main-container">
    <div class="sidebar" id="subtab-sidebar">
      <div class="subtabs" id="subtabs-innova" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmww1')">Asset Class Views</a>
        </div>
      </div>
      <div class="subtabs" id="subtabs-dmw" style="display: none;">
        <div class="list-group">
          <a class="list-group-item" onclick="showContent('dmw1')">Table Format</a>
        </div>
      </div>
    </div>
    <div class="content" id="content-area">
      <div id="landing" class="content-pane">
        <h2>Welcome Innova Team</h2>
        <p>Use the tabs above to navigate sections. Today is: {{today_date}}</p>
        <ul>
          <li>{{crsp_dimensional}}</li>
        </ul>
      </div>
      <div id="dmww1" class="content-pane" style="display:none;">
        <div>{eur_daily}</div>
      </div>
      <div id="dmw1" class="content-pane" style="display:none;">
        <div>{{equity_styled}}</div>
      </div>
    </div>
  </div>

  <script src="https://code.jquery.com/jquery-3.3.1.slim.min.js"></script>
  <script src="https://cdnjs.cloudflare.com/ajax/libs/popper.js/1.14.7/umd/popper.min.js"></script>
  <script src="https://stackpath.bootstrapcdn.com/bootstrap/4.3.1/js/bootstrap.min.js"></script>

  <script>
    function showContent(contentId) {{
      const panes = document.getElementsByClassName('content-pane');
      for (let i = 0; i < panes.length; i++) {{
        panes[i].style.display = 'none';
      }}

      const el = document.getElementById(contentId);
      if (el) {{
        el.style.display = 'block';

        setTimeout(() => {{
          const svgs = el.querySelectorAll('svg');
          svgs.forEach(svg => {{
            const container = svg.closest('div');
            if (typeof Plotly !== 'undefined' && container) {{
              Plotly.Plots.resize(container);
            }}
          }});
        }}, 300);
      }}
    }}

    function showMainTab(tabId) {{
      const subtabs = document.getElementsByClassName('subtabs');
      for (let i = 0; i < subtabs.length; i++) {{
        subtabs[i].style.display = 'none';
      }}

      const currentSubtab = document.getElementById('subtabs-' + tabId);
      if (currentSubtab) {{
        currentSubtab.style.display = 'block';
      }}

      showContent('landing');

      const mainTabLinks = document.querySelectorAll('.nav-tabs .nav-link');
      mainTabLinks.forEach(link => link.classList.remove('active'));

      const activeLink = document.querySelector('.nav-tabs .nav-link[data-main-tab=\"' + tabId + '\"]');
      if (activeLink) {{
        activeLink.classList.add('active');
      }}
    }}

    window.onload = function () {{
      showMainTab('innova');
      showContent('landing');
    }};
  </script>
</body>
</html>
"""

# Write the HTML to a file
with open("github/index.html", "w") as file:
    file.write(html_template)

In [21]:
# testing_morningstar_data.py
import os
import morningstar_data as md

os.environ['MD_AUTH_TOKEN']="eyJhbGciOiJSUzI1NiIsInR5cCI6IkpXVCIsImtpZCI6Ik1EY3hOemRHTnpGRFJrSTRPRGswTmtaRU1FSkdOekl5TXpORFJrUTROemd6TWtOR016bEdOdyJ9.eyJodHRwczovL21vcm5pbmdzdGFyLmNvbS9lbWFpbCI6ImpiaWxzZWxAaW5ub3ZhYW0uY29tLmF1IiwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vcm9sZSI6WyJCVFRGLkludmVzdG1lbnREYXRhU291cmNlXzN4IiwiRW5hYmxlZCBBbmFseXRpY3MgTGFiIERlbGl2ZXJ5IE5vdGVib29rcyIsIkZpcm0tbGV2ZWwgQXNzZXQgQ2xhc3NpZmljYXRpb24iLCJEaXNhYmxlIERlZmluZWQgQ29udHJpYnV0aW9uIFBsYW5zIiwiQ3VzdG9tIEZpcm0gU2VjdXJpdHkgRGF0YSBQb2ludCBNYXN0ZXIgSW1wb3J0ZXIiLCJQb3J0Zm9saW8gQW5hbHlzaXMgVXNlciIsIlBlcnNvbmEuRGlyZWN0Rm9yQXNzZXRNYW5hZ2VtZW50IiwiTGljZW5zZS5QcmVzZW50YXRpb25TdHVkaW8iXSwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vY29tcGFueV9pZCI6ImRmZDAxYzEzLTU1ZTMtNDBiMy1hMzQ2LTcwNmZmMTg5MDFiMiIsImh0dHBzOi8vbW9ybmluZ3N0YXIuY29tL2xlZ2FjeV9jb21wYW55X2lkIjoiZGZkMDFjMTMtNTVlMy00MGIzLWEzNDYtNzA2ZmYxODkwMWIyIiwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vcm9sZV9pZCI6WyIxNDUxMzIwOC1mNmVhLTQzYjMtOTYwMi0wMjQxOGE1NjBhOWYiLCI3OGJhMWFlNy0xZWUzLTQ0YTAtYTAxOC0wOGM1NThmZWNmMTciLCJkYzAzMmIzMy01Nzg3LTQ3MzYtODQ5ZS00NjA3NmZjNjlhZmYiLCI4MjYyOWNkMC1kZjgwLTRlNWMtYjNiYS02YmQyNWU5MzBhNDIiLCI1NmQ4ZDMzNS1iYjRkLTQ4ZjgtODY0Zi1kNTQyNjYxZTM1ZTIiLCJkYzMxN2Q5OC0xMTAwLTQyM2YtOTUzZi1mZjRkYjc4MzUwMTgiXSwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vcHJvZHVjdCI6WyJESVJFQ1QiLCJQUyJdLCJodHRwczovL21vcm5pbmdzdGFyLmNvbS9jb21wYW55IjpbeyJpZCI6ImRmZDAxYzEzLTU1ZTMtNDBiMy1hMzQ2LTcwNmZmMTg5MDFiMiIsInByb2R1Y3QiOiJESVJFQ1QifV0sImh0dHBzOi8vbW9ybmluZ3N0YXIuY29tL21zdGFyX2lkIjoiRDg2RUUzOTgtQTBCQy00MkVBLTkzOUYtRjJGQjIwMDlENUU4IiwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vZW1haWxfdmVyaWZpZWQiOnRydWUsImh0dHBzOi8vbW9ybmluZ3N0YXIuY29tL3Bhc3N3b3JkQ2hhbmdlUmVxdWlyZWQiOmZhbHNlLCJodHRwczovL21vcm5pbmdzdGFyLmNvbS91aW1fcm9sZXMiOiJNRF9NRU1CRVJfMV8xLERJUkVDVCxBVUFSQ19VU0VSIiwiaHR0cHM6Ly9tb3JuaW5nc3Rhci5jb20vbGFzdF9wYXNzd29yZF9yZXNldCI6IjIwMjQtMDYtMjFUMDI6MjI6MzAuODE4WiIsImlzcyI6Imh0dHBzOi8vbG9naW4tcHJvZC5tb3JuaW5nc3Rhci5jb20vIiwic3ViIjoiYXV0aDB8RDg2RUUzOTgtQTBCQy00MkVBLTkzOUYtRjJGQjIwMDlENUU4IiwiYXVkIjpbImh0dHBzOi8vdWltLXByb2QubW9ybmluZ3N0YXIuYXV0aDAuY29tL2FwaS92Mi8iLCJodHRwczovL3VpbS1wcm9kLm1vcm5pbmdzdGFyLmF1dGgwLmNvbS91c2VyaW5mbyJdLCJpYXQiOjE3NDEyMzQwMzgsImV4cCI6MTc0MTMyMDQzOCwic2NvcGUiOiJvcGVuaWQgcHJvZmlsZSBlbWFpbCBvZmZsaW5lX2FjY2VzcyIsImF6cCI6IkNoZUtNNHVqOGpQVDYwYVUySTRjUGxIOHJES2RPc1pkIn0.uzkc4RTfPfyHwEtrzzViqIJ2rBnWQE8s_kVOIB7O-9hVymsylfIQHjFjrH82C_Wa5qa_KzGRKpY-XgVVEIrnARefWgOs_p0oh0hX6A7js9IK4_oYP2z_XDDDJQho_yTmW_7qhgrC56QllAsaDOtWE78qBOjxBHO4479I3BMdFU3xg4V6faOo17IsvITBt_xKp0SdB6H8LPX1aJ7uWE2I8ELNqs3576-vungu-ZGuOfYI685HkTHBR99HKag0BULef71EJeoeHbjnaCInUGdq5dX8OEP7g70zPfu24hHyjU26LhYOKZcPQybsoElYB-qE8_KIlY5OqvduwSrlcy5Sng"

In [ ]:
timeseries_settings = md.direct.get_data_point_settings(data_point_ids=["42"])
portfolio_data = md.direct.portfolio.get_data(portfolio_id='72e6738b-53d3-49a1-97dd-1c95de8afb3b;MD', data_point_settings = timeseries_settings, start_date="2022-12-31", end_date="2023-01-31")

In [ ]:
portfolio_data.T

In [ ]:
df = md.direct.user_items.get_portfolios()
filtered_df = df[
    (df['Name'].str.contains('Hub24', case=False, na=False)) &  # Keep 'Hub24' (case insensitive)
    (~df['Name'].str.contains('No', case=False, na=False)) &    # Exclude 'No' or 'no'
    (df['Type'] == 'model_portfolios')                          # Keep only 'model_portfolios'
]
classification_model_ports = filtered_df.set_index('Name')

In [ ]:
md.direct.portfolio.get_data_sets()['name'].to_list()

In [ ]:
md.direct.portfolio.get_data(portfolio_id = porto, data_set_id="3" ,                               
                                           start_date="2022-04-30",
                                           end_date="2025-03-01", currency="AUD")

In [ ]:
porto = '72e6738b-53d3-49a1-97dd-1c95de8afb3b;MD'
#timeseries_settings = md.direct.get_data_point_settings(data_point_ids=["3"])
return_test = md.direct.portfolio.get_data(portfolio_id = porto, data_set_id="3" ,                               
                                           start_date="2022-04-30",
                                           end_date="2025-03-01", currency="AUD")


for fund_id in classification_model_ports['PortfolioId']:
    holder = md.direct.portfolio.get_data(portfolio_id = fund_id, data_set_id="3" ,                               
                                            start_date="2022-04-30",
                                            end_date="2025-03-01", currency="AUD")

AASHNA




In [ ]:
cpi_AUS = blp.bdh(tickers = 'AUCPIYOY Index', flds = 'px_last', start_date = '01-01-2019', Per = 'M')
cpi_AUS

In [ ]:
cpi_AUS = blp.bdh(tickers = 'AUCPIYOY Index', flds = 'px_last', start_date = '01-01-2019', Per = 'M').droplevel(axis=1,level=1)
cpi_AUS

In [ ]:
cpi_AUS_graph = px.line(cpi_AUS)
cpi_AUS_graph.update_layout(
    font_family="Montserrat",
    title={
        "text": 'This is where the title goes!',
        "font": {"size": 22}
    },
    xaxis_title="Date",
    yaxis_title="Price",
    yaxis=dict(side="left",
        title="Price",
        titlefont=dict(color="black"),
        tickfont=dict(color="black"),
        gridcolor="#ECECEC",
        linecolor="#ECECEC",
    ),
    plot_bgcolor="white",
    paper_bgcolor="white",
    xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
    width=950,
    height=600,
    legend=dict(
        orientation="h",
        y=-0.075,
        x=0.5,
        xanchor="center"
    )
)
 
color_palette = [
    "#30415f",  # Flame
    "#778BA5",  # Amber
    "#87b1a1",  # Sage
    "#5ac5fe",  #
    "#a8c686",  # Pistachio
    "#a0a197",  # Slate
    "#30415f",  # Midnight
    "#2337C6",  #
    "#B7B1B0",  # Taupe
    "#778BA5",  # Cadet Blue
    "#990000"   #
]
 
for i, trace in enumerate(cpi_AUS_graph.data):
    trace.update(line=dict(color=color_palette[i % len(color_palette)]))
 
cpi_AUS_graph

In [ ]:
def fetch_currency_data():
    
    currency_pairs = [
        'AUDUSD Curncy', 'AUDJPY Curncy', 'AUDCAD Curncy', 'AUDCHF Curncy', 
        'AUDNZD Curncy', 'AUDEUR Curncy', 'AUDKRW Curncy', 'AUDGBP Curncy', 
        'AUDHKD Curncy'
    ]
    
    
    currency_data = blp.bdh(
        tickers=currency_pairs,
        flds='PX_LAST', 
        start_date='01-01-2019', 
        Per='M'  
    )
    
    
    currency_data = currency_data.droplevel(axis=1, level=1)
    
    return currency_data

def normalize_data(currency_data):
   
    normalized_data = currency_data / currency_data.iloc[0]  
    return normalized_data

def plot_currency_data(currency_data, title):
    
    normalized_data = normalize_data(currency_data)
    
    
    normalized_data = normalized_data.fillna(method='ffill')

    
    currency_graph = px.line(normalized_data, title=title)
    
   
    currency_graph.update_layout(
        font_family="Montserrat",
        title={
            "text": title,
            "font": {"size": 22}
        },
        xaxis_title="Date",
        yaxis_title="Normalized Exchange Rate",
        yaxis=dict(side="left",
            title="Normalized Exchange Rate",
            titlefont=dict(color="black"),
            tickfont=dict(color="black"),
            gridcolor="#ECECEC",
            linecolor="#ECECEC",
        ),
        plot_bgcolor="white",
        paper_bgcolor="white",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        width=950,
        height=600,
        legend=dict(
            orientation="h",
            y=-0.075,
            x=0.5,
            xanchor="center"
        )
    )
    
    
    currency_graph.show()

currency_data = fetch_currency_data()


plot_currency_data(currency_data, "AUD Currency Pairs")

In [ ]:
def fetch_aud_usd_data():
    tickers = ['AUDUSD Curncy']  
    start_date = '07-01-2023'     
    end_date = '13-03-2025'       

    data = blp.bdh(
        tickers=tickers,
        flds='PX_LAST',  
        start_date=start_date,
        end_date=end_date,
        Per='D'  
    )
    
    data = data.droplevel(axis=1, level=1)
    
    return data

def calculate_moving_averages(data):
    data['50_MA'] = data['AUDUSD Curncy'].rolling(window=50).mean()
    data['200_MA'] = data['AUDUSD Curncy'].rolling(window=200).mean()
    return data

def plot_aud_usd(data):
    fig = go.Figure()

    # Set the color for each trace to '#30415f'
    fig.add_trace(go.Scatter(x=data.index, y=data['AUDUSD Curncy'], mode='lines', name='AUD/USD', line=dict(color="#30415f")))

    fig.add_trace(go.Scatter(x=data.index, y=data['50_MA'], mode='lines', name='50-Day MA', line=dict(color="#30415f")))

    fig.add_trace(go.Scatter(x=data.index, y=data['200_MA'], mode='lines', name='200-Day MA', line=dict(color="#30415f")))

    fig.update_layout(
        title="AUD/USD with 50-Day and 200-Day Moving Averages",
        xaxis_title="Date",
        yaxis_title="AUD/USD Exchange Rate",
        font_family="Montserrat",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        yaxis=dict(side="left", gridcolor="#ECECEC", linecolor="#ECECEC"),
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=950,
        height=600,
        legend=dict(
            orientation="h",
            y=-0.075,
            x=0.5,
            xanchor="center"
        )
    )

    fig.show()

aud_usd_data = fetch_aud_usd_data()
aud_usd_data_with_mas = calculate_moving_averages(aud_usd_data)
plot_aud_usd(aud_usd_data_with_mas)

In [ ]:
def fetch_cpi_yoy_data():
   
    tickers = ['CNCPIYOY Index', 'JNCPIYOY Index', 'CPI YOY Index', 'UKRPCJYR Index']
    start_date = '01-01-2008'  
    end_date = '01-01-2025'    

    
    data = blp.bdh(
        tickers=tickers,
        flds='PX_LAST',  
        start_date=start_date,
        end_date=end_date,
        Per='M'  
    )

    
    if data.columns.nlevels > 1:
        data = data.droplevel(axis=1, level=1)
    
    return data


def plot_cpi_yoy(data):
    
    if data.empty:
        print("No data available for the requested date range.")
        return  
    
    
    countries = ['China', 'Japan', 'United States', 'United Kingdom']

    
    fig = go.Figure()

    
    for i, ticker in enumerate(data.columns):
        fig.add_trace(go.Scatter(
            x=data.index,  
            y=data[ticker],  
            mode='lines',  
            name=countries[i],  
            line=dict(color="#30415f" if i == 0 else "#7F8C8D") 
        ))

    
    fig.update_layout(
        title="CPI YoY for Different Countries (2008-2025)",
        xaxis_title="Date",
        yaxis_title="CPI YoY (%)",
        font_family="Montserrat",
        xaxis=dict(gridcolor="#ECECEC", linecolor="#ECECEC"),
        yaxis=dict(side="left", gridcolor="#ECECEC", linecolor="#ECECEC"),
        plot_bgcolor="white",
        paper_bgcolor="white",
        width=950,
        height=600,
        legend=dict(
            orientation="h",
            y=-0.1,
            x=0.5,
            xanchor="center"
        )
    )

    fig.show()


def main():
    
    cpi_yoy_data = fetch_cpi_yoy_data()

    
    plot_cpi_yoy(cpi_yoy_data)


if __name__ == '__main__':
    main()

In [ ]:
import blpapi
from xbbg import blp
import pandas as pd
import numpy as np
from scipy.stats.mstats import winsorize
import plotly.graph_objects as go
from datetime import datetime

# Reference dates
date_mar = pd.Timestamp.today()
date_feb = pd.Timestamp.today() - pd.DateOffset(months=1)

# Grouped assets
groups = {
    "Bonds": [
        {"name": "US Treasuries", "ticker": "LGTRTRUU Index", "field": "yield_to_worst"}, 
        {"name": "US HY OAS", "ticker": "LF98OAS Index", "field": "px_last"}, 
        {"name": "US Inv Grade Credit OAS", "ticker": "LUCROAS Index", "field": "px_last"},
        {"name": "AUS Corp Bond Comp", "ticker": "BACR0 Index", "field": "OAS_SPREAD_MID"},
        {"name": "Ausbond Credit FRN", "ticker": "BAFRN0 Index", "field": "OAS_SPREAD_MID"},
        {"name": "Euro IG OAS", "ticker": "LECPOAS Index", "field": "px_last"}
    ],
    "Equities": [
        {"name": "MSCI World", "ticker": "MXWO Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
        {"name": "MSCI World Value", "ticker": "MXWO000V Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
        {"name": "MSCI World Growth", "ticker": "MXWO000G Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
        {"name": "MSCI EM", "ticker": "MXEF Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
        {"name": "MSCI World Small Cap", "ticker": "MXWOSC Index", "field": "LONG_TERM_PRICE_EARNINGS_RATIO"},
    ],
    "Real Assets": [
        {"name": "ASX 200 A-Reit", "ticker": "AS51PROP Index", "field": "px_to_tang_bv_per_sh"},
        {"name": "FTSE EORA/NAREIT Dev", "ticker": "ENGL Index", "field": "px_to_tang_bv_per_sh"},
        {"name": "S&P Global Infra", "ticker": "SPGTIND Index", "field": "px_to_tang_bv_per_sh"},
    ]
}

# Flatten asset list while preserving group info
assets = []
for group, asset_list in groups.items():
    for asset in asset_list:
        asset['group'] = group
        assets.append(asset)

# Date range
start_date = (pd.Timestamp.today() - pd.DateOffset(years=25)).strftime('%Y-%m-%d')
end_date = pd.Timestamp.today().strftime('%Y-%m-%d')

# Helper function
def get_zscore_info(ticker, field, target_date):
    df = blp.bdh(ticker, flds=field, start_date=start_date, end_date=end_date)
    df.columns = ['value']
    df.dropna(inplace=True)
    if df.empty:
        return None, None, None

    first_valid = df.index[0].strftime('%b %d, %Y')

    values_winsor = winsorize(df['value'], limits=[0.01, 0.01])
    mean_val = np.mean(values_winsor)
    std_val = np.std(values_winsor)
    z_scores = (values_winsor - mean_val) / std_val
    df['z_score'] = z_scores
    df['date'] = df.index

    try:
        idx = df.index.get_loc(target_date, method='nearest')
        z = df.iloc[idx]['z_score']
        label = df.index[idx].strftime('%b %d, %Y')
        return z, label, first_valid
    except:
        return None, None, first_valid

# Build x-axis labels
x_tickvals = []
x_ticktexts = []
fig = go.Figure()

# Legend control
legend_added = {f"Today - {date_mar}": False, "1 Month Ago": False}

# Draw background bands
for i, asset in enumerate(assets):
    x0, x1 = i - 0.4, i + 0.4
    fig.add_shape(type="rect", x0=x0, x1=x1, y0=2, y1=3, fillcolor="gainsboro", opacity=0.2, layer="below", line_width=0)
    fig.add_shape(type="rect", x0=x0, x1=x1, y0=-3, y1=-2, fillcolor="gainsboro", opacity=0.2, layer="below", line_width=0)
    fig.add_shape(type="rect", x0=x0, x1=x1, y0=1, y1=2, fillcolor="lightgray", opacity=0.3, layer="below", line_width=0)
    fig.add_shape(type="rect", x0=x0, x1=x1, y0=-2, y1=-1, fillcolor="lightgray", opacity=0.3, layer="below", line_width=0)
    fig.add_shape(type="rect", x0=x0, x1=x1, y0=-1, y1=1, fillcolor="gray", opacity=0.3, layer="below", line_width=0)

# Add markers
for i, asset in enumerate(assets):
    z_mar, label_mar, first_valid = get_zscore_info(asset['ticker'], asset['field'], date_mar)
    z_feb, label_feb, _ = get_zscore_info(asset['ticker'], asset['field'], date_feb)

    # Invert if yield_to_worst
    if asset['field'] == 'yield_to_worst':
        if z_mar is not None: z_mar *= -1
        if z_feb is not None: z_feb *= -1

    # Tick label with field and first valid date
    label_html = f"{asset['name']}<br><span style='font-size:11px;color:gray'>{asset['field']}<br>{first_valid if first_valid else '—'}</span>"
    x_tickvals.append(asset['name'])
    x_ticktexts.append(label_html)

    if z_feb is not None and z_mar is not None and z_feb == z_mar:
        z_feb += 0.05

    if z_mar is not None:
        fig.add_trace(go.Scatter(
            x=[asset['name']],
            y=[z_mar],
            mode='markers+text',
            name="March – Current" if not legend_added["mar"] else None,
            marker=dict(symbol='circle', size=14, color="#30415f"),
            text=[f"{z_mar:.2f}"],
            textposition='top center',
            showlegend=not legend_added["mar"],
            textfont=dict(family="Montserrat")
        ))
        legend_added["mar"] = True

    if z_feb is not None:
        fig.add_trace(go.Scatter(
            x=[asset['name']],
            y=[z_feb],
            mode='markers+text',
            name="Feb – Prev Month" if not legend_added["feb"] else None,
            marker=dict(symbol='triangle-down', size=14, color="#a8c686"),
            text=[f"{z_feb:.2f}"],
            textposition='bottom center',
            showlegend=not legend_added["feb"],
            textfont=dict(family="Montserrat")
        ))
        legend_added["feb"] = True

# Final layout
fig.update_layout(
    title={
        'text': 'Asset Class Valuations (Z-scores based on 25-year average valuation measures)',
        'font': {
            'family': 'Montserrat',
            'size': 18
        }
    },
    yaxis=dict(
        title={
            'text': 'Z-score',
            'font': {
                'family': 'Montserrat',
                'size': 14
            }
        },
        range=[-3.5, 3],
        zeroline=True,
        zerolinewidth=2,
        zerolinecolor='black',
        showgrid=False,
        tickfont=dict(family="Montserrat")
    ),
    xaxis=dict(
        tickvals=x_tickvals,
        ticktext=x_ticktexts,
        showgrid=False,
        tickfont=dict(family="Montserrat")
    ),
    plot_bgcolor='white',
    showlegend=True,
    legend=dict(
        orientation="h",
        yanchor="top",
        y=-0.2,
        xanchor="center",
        x=0.5,
        font=dict(family="Montserrat")
    ),
    height=720,
    width=max(900, 150 + 100 * len(x_tickvals)),
    margin=dict(t=80, b=160),
    font=dict(family="Montserrat")
)
aashna_all_asset_class_z_score_valuations = fig
aashna_all_asset_class_z_score_valuations_html = plot(aashna_all_asset_class_z_score_valuations, output_type='div', include_plotlyjs='cdn')